# Robert Smith, Gencer Ates
'''
We declare that all work submitted for final exam in this report is our own work and does not involve plagiarism
or collaboration outside of our own team. We also declare that we have not discussed the problems and
solutions with anyone.'''

In [1]:
'''
Sources:
https://www.kaggle.com/cdc/national-health-and-nutrition-examination-survey
https://stackoverflow.com/questions/34077353/how-to-change-dataframe-column-names-in-pyspark
'''

'\nSources:\nhttps://www.kaggle.com/cdc/national-health-and-nutrition-examination-survey\nhttps://stackoverflow.com/questions/34077353/how-to-change-dataframe-column-names-in-pyspark\n'

In [2]:
import findspark
findspark.init()
import pyspark 
from pyspark import SparkContext
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 

In [3]:
import os
port = 4040 + hash(os.getcwd().split("/")[2])%200
config = pyspark.SparkConf().set('spark.executor.memory', '512M')\
            .set('spark.cores.max', '4')\
            .set('spark.port.maxRetries','200')\
            .set('spark.ui.port', port)\
            .set('spark.master', 'spark://polyp1:7077')\
            .set('spark.app.name', "legDay") 
print port,     os.getcwd().split("/")[2]
#sc = pyspark.SparkContext(appName="Project 1", master='spark://polyp1:7077',conf = config )

4147 oga219


In [4]:
ss = pyspark.sql.SparkSession.builder.config(conf = config).getOrCreate()

print ss.conf.get('spark.app.name')
print ss.conf.get('spark.master')

#### Input data
sc = ss.sparkContext
examinationRDD = sc.textFile("/scratch/ISE495/2018_project_02/NHANES/examination.csv")
demographicRDD = sc.textFile("/scratch/ISE495/2018_project_02/NHANES/demographic.csv")
otherRDD = sc.textFile("/scratch/ISE495/2018_project_02/NHANES/other.csv")

legDay
spark://polyp1:7077


# Task 1.1: preparing the data for SQL


In [5]:
df3 = ss.read.csv("/scratch/ISE495/2018_project_02/NHANES/examination.csv", header = True ,inferSchema = True) #224 columns
df1 = ss.read.csv("/scratch/ISE495/2018_project_02/NHANES/demographic.csv", header = True ,inferSchema = True) #47 columns
# df2 = ss.read.csv("/scratch/ISE495/2018_project_02/NHANES/diet.csv", header = True ,inferSchema = True) #168 columns
# df4 = ss.read.csv("/scratch/ISE495/2018_project_02/NHANES/labs.csv", header = True ,inferSchema = True) #424 columns
# df5 = ss.read.csv("/scratch/ISE495/2018_project_02/NHANES/medications.csv", header = True ,inferSchema = True) #13 columns
df6 = ss.read.csv("/scratch/ISE495/2018_project_02/NHANES/questionnaire.csv", header = True ,inferSchema = True) #953 columns

In [6]:
# df6.show()

In [7]:
df = df3.join(df1, [ "SEQN" ])
'''
    #not using medication becaus it has multiples of same
''' 
df = df.dropDuplicates(subset=None)

# Task 1.1.1: Understand the Dataframe

In [8]:
# print df.dtypes #matches column names with data types within columns
# print df #shows column names with data types within columns
# print df.show()
# print df.explain(True)  #explains logical & physical plans
# print len(df.columns)


# Task 1.2: divide df into veteran vs. non-veteran

In [9]:
df.createOrReplaceTempView("kingDF")

veteranDF = ss.\
    sql("FROM kingDF WHERE DMQMILIZ=1")
civilianDF = ss.\
    sql("FROM kingDF WHERE DMQMILIZ=2")

In [10]:
veteranDF.createOrReplaceTempView("veteranDFsql")
veteranDFtest = ss.\
    sql("SELECT DMQMILIZ FROM veteranDFsql WHERE DMQMILIZ=2")

#veteranDFtest.show() --> 
# +--------+
# |DMQMILIZ|
# +--------+
# +--------+
# works out!

In [11]:
# print (2016 - 1982) + 18 # 1972 # # print (1973+1991)/2 = 1982 # 2016 --> #52 is the cut-off age for seperation 

# Task 1.3: divide veterans and non-veterans into relevant age groups, and then convert those data frames back into rdd

In [12]:
veteranDF.createOrReplaceTempView("vetsOlderSql")
vetDFolder = ss.\
    sql("FROM vetsOlderSql WHERE RIDAGEYR > 52") #SELECT RIDAGEYR == age > 50 # print vetDFolder.count() --> 386
vetDFyounger = ss.\
    sql("FROM vetsOlderSql WHERE RIDAGEYR <= 52")

In [13]:
# vetDFolder.describe(['RIDAGEYR']).show()
# +-------+-----------------+
# |summary|         RIDAGEYR|
# +-------+-----------------+
# |  count|              386|
# |   mean|70.20466321243524|
# | stddev|8.569779521966169|
# |    min|               53|
# |    max|               80|
# +-------+-----------------+

# vetDFyounger.describe(['RIDAGEYR']).show()
# +-------+-----------------+
# |summary|         RIDAGEYR|
# +-------+-----------------+
# |  count|              137|
# |   mean|40.08759124087591|
# | stddev|9.410159132050369|
# |    min|               19|
# |    max|               52|
# +-------+-----------------+

In [14]:
# we are only using the corresponding age ranges of the civilians to compare to veterans
civilianDF.createOrReplaceTempView("CivisSql")
civiDFolder = ss.\
    sql("FROM CivisSql WHERE RIDAGEYR > 52") #SELECT RIDAGEYR == age > 50 # print vetDFolder.count() --> 386
civiDFyounger = ss.\
    sql("FROM CivisSql WHERE RIDAGEYR > 19 AND RIDAGEYR < 53")

In [15]:
#convert back to RDD

vetRddOlder = vetDFolder.rdd
vetRddYounger = vetDFyounger.rdd

civiRddOlder = civiDFolder.rdd
civiRddYounger = civiDFyounger.rdd

In [16]:
print civiRddOlder

MapPartitionsRDD[74] at javaToPython at NativeMethodAccessorImpl.java:0


# Task 2: Do Dimensionality Reduction on identified important data, then normalize that data

In [17]:
#we performed this task by hand-selecting data that seemed most important from the most valuable files

keep_list_examination = ["BPXSY1","BPXDI1","BMXBMI","OHDEXSTS"]
# == ["bloodPressure_systolic","bloodPressure_dyastolic","weight","height_cm","BMI","oralHealthStatus"]
keep_list_demographics = ["SEQN","DMDEDUC2","DMDFMSIZ","DMDHHSIZ",\
                          "DMDHHSZE","RIAGENDR", "DMDMARTL", "INDFMPIR"]
# == ["ID","highestGradeSchool","totalPeopleInFamily","totalPeopleInHouse"]\
#    [ "numAdultsinHouse","gender","maritalStatus","ratioIncometoPoverty]

# keep_list_examination = ["BPXSY1 as bloodPressure_systolic","BPXDI1 as bloodPressure_dyastolic","BMXWT as weight","BMXHT as height_cm","BMXBMI as BMI","OHDEXSTS as oralHealthStatus"]
# keep_list_demographics = ["SEQN as ID","DMDEDUC2 as highestGradeSchool","DMDFMSIZ as totalPeopleInFamily","DMDHHSIZ as totalPeopleInHouse","DMDHHSZA as numChildren1","DMDHHSZB as numChildren2",\
#                           "DMDHHSZE as numAdultsinHouse","RIAGENDR as gender", "DMDMARTL as maritalStatus", "INDFMPIR as ratioIncometoPoverty"]
keep_list_medcolsToAdd = ["DPQ010","DPQ020","DPQ030"\
                                          ,"DPQ040","DPQ050","DPQ060","DPQ070","DPQ080","DPQ090"]

In [18]:
# 1.1 merging columns together for data normalization

# select columns (of kids, and fill null values with 0)

colsToAdd = ["SEQN","DMDHHSZA","DMDHHSZB"]
dfForKids = vetDFolder.select([column for column in df.columns if column in colsToAdd])\
.na.fill(0)

dfForKids1 = vetDFyounger.select([column for column in df.columns if column in colsToAdd])\
.na.fill(0)

dfForKids2 = civiDFolder.select([column for column in df.columns if column in colsToAdd])\
.na.fill(0)

dfForKids3 = civiDFyounger.select([column for column in df.columns if column in colsToAdd])\
.na.fill(0)

In [19]:
#combine kids columns into one column

dfForKids_total = dfForKids.withColumn('totalKids', (dfForKids.DMDHHSZA + dfForKids.DMDHHSZB)).drop('DMDHHSZA',"DMDHHSZB")\
# .fillna(_average_,subset=["column name"]) -- for replacing with average instead of null
# dfForKids_total.show(4)
# +-----+---------+
# | SEQN|totalKids|
# +-----+---------+
# |80900|        0|
# |82949|        0|
# |83684|        0|
# |79201|        1|
# +-----+---------+
# only showing top 4 rows


dfForKids_total1 = dfForKids1.withColumn('totalKids', (dfForKids1.DMDHHSZA + dfForKids1.DMDHHSZB)).drop('DMDHHSZA',"DMDHHSZB")
# dfForKids_total1.show(4)
# +-----+---------+
# | SEQN|totalKids|
# +-----+---------+
# |82359|        0|
# |77095|        0|
# |77682|        1|
# |78963|        0|
# +-----+---------+
# only showing top 4 rows

dfForKids_total2 = dfForKids2.withColumn('totalKids', (dfForKids2.DMDHHSZA + dfForKids2.DMDHHSZB)).drop('DMDHHSZA',"DMDHHSZB")
# dfForKids_total2.show(4)
# +-----+---------+
# | SEQN|totalKids|
# +-----+---------+
# |73723|        0|
# |75860|        0|
# |76395|        0|
# |76932|        0|
# +-----+---------+
# only showing top 4 rows

dfForKids_total3 = dfForKids3.withColumn('totalKids', (dfForKids3.DMDHHSZA + dfForKids3.DMDHHSZB)).drop('DMDHHSZA',"DMDHHSZB")
# dfForKids_total3.show(4)
# +-----+---------+
# | SEQN|totalKids|
# +-----+---------+
# |74498|        2|
# |74551|        0|
# |75274|        0|
# |75328|        0|
# +-----+---------+
# only showing top 4 rows

In [20]:
# 1.2 replacing null values of certain columns with their averages

# find averages of specified health-related columns within each of the four datasets

fIt = vetDFolder.describe("BPXSY1", "BPXDI1", "BMXBMI").rdd.map(tuple)
def myDude(rdd):
    summary = rdd[0]
    bloodPressure_systolic = float(rdd[1])
    bloodPressure_dyastolic = float(rdd[2])
    BMI = float(rdd[3])
    return (summary, bloodPressure_systolic, bloodPressure_dyastolic, BMI)
myDuderdd= fIt.map(myDude).collect()
avgSys = myDuderdd[1][1]
avgDys = myDuderdd[1][2]
avgBMI = myDuderdd[1][3]
print "0:", avgSys, avgDys, avgBMI
# 131.670391061 67.7877094972 28.888976378

0: 131.670391061 67.7877094972 28.888976378


In [21]:
fIt = vetDFyounger.describe("BPXSY1", "BPXDI1", "BMXBMI").rdd.map(tuple)
myDuderdd= fIt.map(myDude).collect()
avgSys1 = myDuderdd[1][1]
avgDys1 = myDuderdd[1][2]
avgBMI1 = myDuderdd[1][3]
print "1:",avgSys1, avgDys1, avgBMI1
# 119.59375 73.015625 28.6614814815

1: 119.59375 73.015625 28.6614814815


In [22]:
fIt = civiDFolder.describe("BPXSY1", "BPXDI1", "BMXBMI").rdd.map(tuple)
myDuderdd= fIt.map(myDude).collect()
avgSys2 = myDuderdd[1][1]
avgDys2 = myDuderdd[1][2]
avgBMI2 = myDuderdd[1][3]
print "2:",avgSys2, avgDys2, avgBMI2
# 131.767777778 69.6444444444 29.3429221436

2: 131.767777778 69.6444444444 29.3429221436


In [23]:
fIt = civiDFyounger.describe("BPXSY1", "BPXDI1", "BMXBMI").rdd.map(tuple)
myDuderdd= fIt.map(myDude).collect()
avgSys3 = myDuderdd[1][1]
avgDys3 = myDuderdd[1][2]
avgBMI3 = myDuderdd[1][3]
print "3:",avgSys3, avgDys3, avgBMI3
# 116.768849558 70.2194690265 28.9927296761

3: 116.768849558 70.2194690265 28.9927296761


In [24]:
# task 1.3 isolate self-identified important variables per each of the 4 datasets

#list of things that we need to subset in each of the 4 datasets

keep_list = keep_list_demographics + keep_list_examination #+ keep_list_medcolsToAdd
print keep_list

['SEQN', 'DMDEDUC2', 'DMDFMSIZ', 'DMDHHSIZ', 'DMDHHSZE', 'RIAGENDR', 'DMDMARTL', 'INDFMPIR', 'BPXSY1', 'BPXDI1', 'BMXBMI', 'OHDEXSTS']


In [25]:
#wrapping up tasks

# 1: reducing entire demographics and examination datasets to a selected columns
# 2: merging with the altered kid count list
# 3: renaming the variables to be more telling of their importance
# 4: we are killing weight and height because they are relative to each other, and replaceable by BMI
# 5: replace nulls within certain columns with their averages for each of the 4 datasets (don't need to do this for kids because it's already done previously)

vetDFOlder_DimensionReduction = vetDFolder.select([column for column in df.columns if column in keep_list])\
.join(dfForKids_total, ["SEQN"])\
.selectExpr("SEQN as ID","DMDEDUC2 as highestGradeSchool","DMDFMSIZ as totalPeopleInFamily","DMDHHSIZ as totalPeopleInHouse","totalKids as totalKids_1",\
           "BPXSY1 as bloodPressure_systolic","BPXDI1 as bloodPressure_dyastolic","BMXBMI as BMI","OHDEXSTS as oralHealthStatus")\
.fillna(avgSys,subset=["bloodPressure_systolic"]).fillna(avgDys,subset=["bloodPressure_dyastolic"]).fillna(avgBMI,subset=["BMI"])

vetDFYounger_DimensionReduction = vetDFyounger.select([column for column in df.columns if column in keep_list])\
.join(dfForKids_total1, ["SEQN"])\
.selectExpr("SEQN as ID","DMDEDUC2 as highestGradeSchool","DMDFMSIZ as totalPeopleInFamily","DMDHHSIZ as totalPeopleInHouse","totalKids as totalKids_1",\
           "BPXSY1 as bloodPressure_systolic","BPXDI1 as bloodPressure_dyastolic","BMXBMI as BMI","OHDEXSTS as oralHealthStatus")\
.fillna(avgSys1,subset=["bloodPressure_systolic"]).fillna(avgDys1,subset=["bloodPressure_dyastolic"]).fillna(avgBMI1,subset=["BMI"])



civiDFOlder_DimensionReduction = civiDFolder.select([column for column in df.columns if column in keep_list])\
.join(dfForKids_total2, ["SEQN"])\
.selectExpr("SEQN as ID","DMDEDUC2 as highestGradeSchool","DMDFMSIZ as totalPeopleInFamily","DMDHHSIZ as totalPeopleInHouse","totalKids as totalKids_1",\
           "BPXSY1 as bloodPressure_systolic","BPXDI1 as bloodPressure_dyastolic","BMXBMI as BMI","OHDEXSTS as oralHealthStatus")\
.fillna(avgSys2,subset=["bloodPressure_systolic"]).fillna(avgDys2,subset=["bloodPressure_dyastolic"]).fillna(avgBMI2,subset=["BMI"])

civiDFYounger_DimensionReduction = civiDFyounger.select([column for column in df.columns if column in keep_list])\
.join(dfForKids_total3, ["SEQN"])\
.selectExpr("SEQN as ID","DMDEDUC2 as highestGradeSchool","DMDFMSIZ as totalPeopleInFamily","DMDHHSIZ as totalPeopleInHouse","totalKids as totalKids_1",\
           "BPXSY1 as bloodPressure_systolic","BPXDI1 as bloodPressure_dyastolic","BMXBMI as BMI","OHDEXSTS as oralHealthStatus")\
.fillna(avgSys3,subset=["bloodPressure_systolic"]).fillna(avgDys3,subset=["bloodPressure_dyastolic"]).fillna(avgBMI3,subset=["BMI"])

In [26]:
# vetDFOlder_DimensionReduction.show(2)
# +-----+------------------+-------------------+------------------+-----------+----------------------+-----------------------+----+----------------+
# |   ID|highestGradeSchool|totalPeopleInFamily|totalPeopleInHouse|totalKids_1|bloodPressure_systolic|bloodPressure_dyastolic| BMI|oralHealthStatus|
# +-----+------------------+-------------------+------------------+-----------+----------------------+-----------------------+----+----------------+
# |79220|                 3|                  4|                 4|          0|                   126|                     80|31.9|               1|
# |82582|                 1|                  2|                 2|          0|                   144|                     78|31.9|               1|
# +-----+------------------+-------------------+------------------+-----------+----------------------+-----------------------+----+----------------+
# only showing top 2 rows

In [27]:
# vetDFYounger_DimensionReduction.show(3)

# +-----+------------------+-------------------+------------------+-----------+----------------------+-----------------------+----+----------------+
# |   ID|highestGradeSchool|totalPeopleInFamily|totalPeopleInHouse|totalKids_1|bloodPressure_systolic|bloodPressure_dyastolic| BMI|oralHealthStatus|
# +-----+------------------+-------------------+------------------+-----------+----------------------+-----------------------+----+----------------+
# |83291|                 4|                  4|                 4|          2|                   119|                     73|30.1|               1|
# |82500|                 5|                  4|                 4|          2|                   130|                     72|35.5|               1|
# |80335|                 3|                  1|                 4|          2|                   128|                     70|28.4|               1|
# +-----+------------------+-------------------+------------------+-----------+----------------------+-----------------------+----+----------------+
# only showing top 3 rows

In [28]:
# civiDFOlder_DimensionReduction.show(2)

# +-----+------------------+-------------------+------------------+-----------+----------------------+-----------------------+----+----------------+
# |   ID|highestGradeSchool|totalPeopleInFamily|totalPeopleInHouse|totalKids_1|bloodPressure_systolic|bloodPressure_dyastolic| BMI|oralHealthStatus|
# +-----+------------------+-------------------+------------------+-----------+----------------------+-----------------------+----+----------------+
# |74058|                 2|                  5|                 5|          0|                   128|                     70|34.2|               1|
# |77422|                 5|                  2|                 2|          0|                   168|                     60|21.1|               1|
# +-----+------------------+-------------------+------------------+-----------+----------------------+-----------------------+----+----------------+
# only showing top 2 rows

In [29]:
# civiDFYounger_DimensionReduction.show(2)

# +-----+------------------+-------------------+------------------+-----------+----------------------+-----------------------+----+----------------+
# |   ID|highestGradeSchool|totalPeopleInFamily|totalPeopleInHouse|totalKids_1|bloodPressure_systolic|bloodPressure_dyastolic| BMI|oralHealthStatus|
# +-----+------------------+-------------------+------------------+-----------+----------------------+-----------------------+----+----------------+
# |74166|                 5|                  5|                 5|          3|                   120|                     76|22.9|               1|
# |74251|                 3|                  4|                 4|          1|                   108|                     72|22.9|               1|
# +-----+------------------+-------------------+------------------+-----------+----------------------+-----------------------+----+----------------+
# only showing top 2 rows

# Task 2.1: 
#1 filter df of medications to the important columns
#2 & #3 join with 4 datasets & reduce to important cols
#4 find average for each mental helath column
#5 replace mental health nulls with average
#6 add all mental health columns together

In [30]:
# 2.1 filter df of medications to the important columns

medcolsToAdd = ["SEQN","DPQ010","DPQ020","DPQ030"\
                                          ,"DPQ040","DPQ050","DPQ060","DPQ070","DPQ080","DPQ090"]

dfForMeds = df6.select([column for column in df6.columns if column in medcolsToAdd])

In [31]:
vetDFolder_seqn = vetDFolder.select("SEQN")
# vetDFolder_seqn.show(4)
vetDFyounger_seqn = vetDFyounger.select("SEQN")
civiDFolder_seqn = civiDFolder.select("SEQN")
civiDFyounger_seqn = civiDFyounger.select("SEQN")

In [32]:
# dfForMeds.show(4)

# +-----+------+------+------+------+------+------+------+------+------+
# | SEQN|DPQ010|DPQ020|DPQ030|DPQ040|DPQ050|DPQ060|DPQ070|DPQ080|DPQ090|
# +-----+------+------+------+------+------+------+------+------+------+
# |73557|     1|     0|     0|     0|     0|     0|     0|     0|     0|
# |73558|     2|     0|     0|     0|     0|     0|     0|     0|     0|
# |73559|     0|     0|     0|     0|     0|     0|     0|     0|     0|
# |73560|  null|  null|  null|  null|  null|  null|  null|  null|  null|
# +-----+------+------+------+------+------+------+------+------+------+
# only showing top 4 rows

In [33]:
# joining mental health records with each type of person

vetDFolder_wMed = vetDFolder.join(dfForMeds,["SEQN"])
# print vetDFolder.show()
vetDFyounger_wMed = vetDFyounger.join(dfForMeds,["SEQN"])
civiDFolder_wMed = civiDFolder.join(dfForMeds,["SEQN"])
civiDFyounger_wMed = civiDFyounger.join(dfForMeds,["SEQN"])
# print civiDFyounger_seqn.show(4)

In [34]:
# Get average value per group of people for replacing nulls 

fIt = vetDFolder_wMed.describe("SEQN","DPQ010","DPQ020","DPQ030"\
                                          ,"DPQ040","DPQ050","DPQ060","DPQ070","DPQ080","DPQ090").rdd.map(tuple)
def myDude(rdd):
    summary = rdd[0]
    o1 = float(rdd[1])
    o2 = float(rdd[2])
    o3 = float(rdd[3])
    o4 = float(rdd[4])
    o5 = float(rdd[5])
    o6 = float(rdd[6])
    o7 = float(rdd[7])
    o8 = float(rdd[8])
    o9 = float(rdd[9])
    return (o1,o2,o3,o4,o5,o6,o7,o8, o9)

myDuderdd= fIt.map(myDude).collect()

avgDPQ010 = myDuderdd[1][0]
avgDPQ020 = myDuderdd[1][1]
avgDPQ030 = myDuderdd[1][2]
avgDPQ040 = myDuderdd[1][3]
avgDPQ050 = myDuderdd[1][4]
avgDPQ060 = myDuderdd[1][5]
avgDPQ070 = myDuderdd[1][6]
avgDPQ080 = myDuderdd[1][7]
avgDPQ090 = myDuderdd[1][8]
# print myDuderdd
print avgDPQ010, avgDPQ020, avgDPQ030, avgDPQ040, avgDPQ050, avgDPQ060, avgDPQ070, avgDPQ080, avgDPQ090

78686.253886 0.466850828729 0.32320441989 0.569060773481 0.709944751381 0.279005524862 0.237569060773 0.290055248619 0.182320441989


In [35]:
fIt = vetDFyounger_wMed.describe("SEQN","DPQ010","DPQ020","DPQ030"\
                                          ,"DPQ040","DPQ050","DPQ060","DPQ070","DPQ080","DPQ090").rdd.map(tuple)

myDuderdd= fIt.map(myDude).collect()

avgDPQ0101 = myDuderdd[1][0]
avgDPQ0201 = myDuderdd[1][1]
avgDPQ0301 = myDuderdd[1][2]
avgDPQ0401 = myDuderdd[1][3]
avgDPQ0501 = myDuderdd[1][4]
avgDPQ0601 = myDuderdd[1][5]
avgDPQ0701 = myDuderdd[1][6]
avgDPQ0801 = myDuderdd[1][7]
avgDPQ0901 = myDuderdd[1][8]
# print myDuderdd
print avgDPQ0101, avgDPQ0201, avgDPQ0301, avgDPQ0401, avgDPQ0501, avgDPQ0601, avgDPQ0701, avgDPQ0801, avgDPQ0901

78594.2992701 0.424242424242 0.409090909091 0.590909090909 0.719696969697 0.439393939394 0.272727272727 0.310606060606 0.219696969697


In [36]:
fIt = civiDFolder_wMed.describe("SEQN","DPQ010","DPQ020","DPQ030"\
                                          ,"DPQ040","DPQ050","DPQ060","DPQ070","DPQ080","DPQ090").rdd.map(tuple)

myDuderdd= fIt.map(myDude).collect()

avgDPQ0102 = myDuderdd[1][0]
avgDPQ0202 = myDuderdd[1][1]
avgDPQ0302 = myDuderdd[1][2]
avgDPQ0402 = myDuderdd[1][3]
avgDPQ0502 = myDuderdd[1][4]
avgDPQ0602 = myDuderdd[1][5]
avgDPQ0702 = myDuderdd[1][6]
avgDPQ0802 = myDuderdd[1][7]
avgDPQ0902 = myDuderdd[1][8]
# print myDuderdd
print avgDPQ0102, avgDPQ0202, avgDPQ0302, avgDPQ0402, avgDPQ0502, avgDPQ0602, avgDPQ0702, avgDPQ0802, avgDPQ0902


78563.0793257 0.486297689414 0.451612903226 0.696611081226 0.813878429263 0.428187197418 0.320236813778 0.337997847147 0.231969860065


In [37]:
fIt = civiDFyounger_wMed.describe("SEQN","DPQ010","DPQ020","DPQ030"\
                                          ,"DPQ040","DPQ050","DPQ060","DPQ070","DPQ080","DPQ090").rdd.map(tuple)
myDuderdd= fIt.map(myDude).collect()

avgDPQ0103 = myDuderdd[1][0]
avgDPQ0203 = myDuderdd[1][1]
avgDPQ0303 = myDuderdd[1][2]
avgDPQ0403 = myDuderdd[1][3]
avgDPQ0503 = myDuderdd[1][4]
avgDPQ0603 = myDuderdd[1][5]
avgDPQ0703 = myDuderdd[1][6]
avgDPQ0803 = myDuderdd[1][7]
avgDPQ0903 = myDuderdd[1][8]
# print myDuderdd
print avgDPQ0103, avgDPQ0203, avgDPQ0303, avgDPQ0403, avgDPQ0503, avgDPQ0603, avgDPQ0703, avgDPQ0803, avgDPQ0903

78731.6220472 0.367706422018 0.328193832599 0.582599118943 0.758810572687 0.393171806167 0.247797356828 0.260279001468 0.143906020558


In [38]:
# making the keep list smaller

keepList_2 = ["SEQN","DPQ020","DPQ030","DPQ040","DPQ050","DPQ060","DPQ070","DPQ080","DPQ090"]

vetDFolder_wMed2 = vetDFolder_wMed.select([column for column in df6.columns if column in keepList_2])
vetDFyounger_wMed2 = vetDFyounger_wMed.select([column for column in df6.columns if column in keepList_2])
civiDFolder_wMed2 = civiDFolder_wMed.select([column for column in df6.columns if column in keepList_2])
civiDFyounger_wMed2 = civiDFyounger_wMed.select([column for column in df6.columns if column in keepList_2])

In [39]:
# print vetDFolder_wMed2.show(2)

# +-----+------+------+------+------+------+------+------+------+------+
# | SEQN|DPQ010|DPQ020|DPQ030|DPQ040|DPQ050|DPQ060|DPQ070|DPQ080|DPQ090|
# +-----+------+------+------+------+------+------+------+------+------+
# |80900|     0|     0|     0|     0|     0|     0|     0|     0|     0|
# |82949|     0|     0|     0|     0|     0|     0|     0|     0|     0|
# +-----+------+------+------+------+------+------+------+------+------+
# only showing top 2 rows

# None

In [40]:
#this should narrow the df down to mental health related ones only
#it should also replace nulls with their repective averages for each data set

vetDFolder_noNull = vetDFolder_wMed.fillna(avgDPQ020,subset=["DPQ020"]).fillna(avgDPQ030,subset=["DPQ030"]).fillna(avgDPQ040,subset=["DPQ040"]).fillna(avgDPQ050,subset=["DPQ050"]).fillna(avgDPQ060,subset=["DPQ060"]).fillna(avgDPQ070,subset=["DPQ070"]).fillna(avgDPQ080,subset=["DPQ080"]).fillna(avgDPQ090,subset=["DPQ0290"])
vetDFyounger_noNull = vetDFyounger_wMed.fillna(avgDPQ0201,subset=["DPQ020"]).fillna(avgDPQ0301,subset=["DPQ030"]).fillna(avgDPQ0401,subset=["DPQ040"]).fillna(avgDPQ0501,subset=["DPQ050"]).fillna(avgDPQ0601,subset=["DPQ060"]).fillna(avgDPQ0701,subset=["DPQ070"]).fillna(avgDPQ0801,subset=["DPQ080"]).fillna(avgDPQ0901,subset=["DPQ0290"])
civiDFolder_noNull = civiDFolder_wMed.fillna(avgDPQ0202,subset=["DPQ020"]).fillna(avgDPQ0302,subset=["DPQ030"]).fillna(avgDPQ0402,subset=["DPQ040"]).fillna(avgDPQ0502,subset=["DPQ050"]).fillna(avgDPQ0602,subset=["DPQ060"]).fillna(avgDPQ0702,subset=["DPQ070"]).fillna(avgDPQ0802,subset=["DPQ080"]).fillna(avgDPQ0902,subset=["DPQ0290"])
civiDFyounger_noNull = civiDFyounger_wMed.fillna(avgDPQ0203,subset=["DPQ020"]).fillna(avgDPQ0303,subset=["DPQ030"]).fillna(avgDPQ0403,subset=["DPQ040"]).fillna(avgDPQ0503,subset=["DPQ050"]).fillna(avgDPQ0603,subset=["DPQ060"]).fillna(avgDPQ0703,subset=["DPQ070"]).fillna(avgDPQ0803,subset=["DPQ080"]).fillna(avgDPQ0903,subset=["DPQ0290"])

In [41]:
# print vetDFolder_noNull.show(2)

# +-----+--------+--------+--------+------+------+------+------+-------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+--------+-----+-----+--------+--------+-------+-------+-----+-----+------+-------+------+------+-------+-------+-------+-------+--------+--------+-------+-------+-------+-------+--------+--------+--------+------+------+------+------+------+------+------+------+------+-------+-------+--------+--------+-------+--------+-------+--------+-------+--------+-------+--------+-------+--------+-------+--------+-------+--------+--------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+------+------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+--------+--------+-------+-------+--------+--------+--------+--------+-------+-------+-------+--------+-------+--------+--------+--------+--------+-------+--------+--------+--------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+-------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+------------+------------+-------+--------+--------+--------+--------+------+------+------+------+------+------+------+------+------+
# | SEQN|PEASCST1|PEASCTM1|PEASCCT1|BPXCHR|BPAARM|BPACSZ|BPXPLS|BPXPULS|BPXPTY|BPXML1|BPXSY1|BPXDI1|BPAEN1|BPXSY2|BPXDI2|BPAEN2|BPXSY3|BPXDI3|BPAEN3|BPXSY4|BPXDI4|BPAEN4|BMDSTATS|BMXWT|BMIWT|BMXRECUM|BMIRECUM|BMXHEAD|BMIHEAD|BMXHT|BMIHT|BMXBMI|BMDBMIC|BMXLEG|BMILEG|BMXARML|BMIARML|BMXARMC|BMIARMC|BMXWAIST|BMIWAIST|BMXSAD1|BMXSAD2|BMXSAD3|BMXSAD4|BMDAVSAD|BMDSADCM|MGDEXSTS|MGD050|MGD060|MGQ070|MGQ080|MGQ090|MGQ100|MGQ110|MGQ120|MGD130|MGQ90DG|MGDSEAT|MGAPHAND|MGATHAND|MGXH1T1|MGXH1T1E|MGXH2T1|MGXH2T1E|MGXH1T2|MGXH1T2E|MGXH2T2|MGXH2T2E|MGXH1T3|MGXH1T3E|MGXH2T3|MGXH2T3E|MGDCGSZ|OHDEXSTS|OHDDESTS|OHXIMP|OHX01TC|OHX02TC|OHX03TC|OHX04TC|OHX05TC|OHX06TC|OHX07TC|OHX08TC|OHX09TC|OHX10TC|OHX11TC|OHX12TC|OHX13TC|OHX14TC|OHX15TC|OHX16TC|OHX17TC|OHX18TC|OHX19TC|OHX20TC|OHX21TC|OHX22TC|OHX23TC|OHX24TC|OHX25TC|OHX26TC|OHX27TC|OHX28TC|OHX29TC|OHX30TC|OHX31TC|OHX32TC|OHX02CTC|OHX03CTC|OHX04CTC|OHX05CTC|OHX06CTC|OHX07CTC|OHX08CTC|OHX09CTC|OHX10CTC|OHX11CTC|OHX12CTC|OHX13CTC|OHX14CTC|OHX15CTC|OHX18CTC|OHX19CTC|OHX20CTC|OHX21CTC|OHX22CTC|OHX23CTC|OHX24CTC|OHX25CTC|OHX26CTC|OHX27CTC|OHX28CTC|OHX29CTC|OHX30CTC|OHX31CTC|OHX02CSC|OHX03CSC|OHX04CSC|OHX05CSC|OHX06CSC|OHX07CSC|OHX08CSC|OHX09CSC|OHX10CSC|OHX11CSC|OHX12CSC|OHX13CSC|OHX14CSC|OHX15CSC|OHX18CSC|OHX19CSC|OHX20CSC|OHX21CSC|OHX22CSC|OHX23CSC|OHX24CSC|OHX25CSC|OHX26CSC|OHX27CSC|OHX28CSC|OHX29CSC|OHX30CSC|OHX31CSC|OHX02SE|OHX03SE|OHX04SE|OHX05SE|OHX07SE|OHX10SE|OHX12SE|OHX13SE|OHX14SE|OHX15SE|OHX18SE|OHX19SE|OHX20SE|OHX21SE|OHX28SE|OHX29SE|OHX30SE|OHX31SE|CSXEXSTS|CSXEXCMT|CSQ245|CSQ241|CSQ260A|CSQ260D|CSQ260G|CSQ260I|CSQ260N|CSQ260M|CSQ270|CSQ450|CSQ460|CSQ470|CSQ480|CSQ490|CSXQUIPG|CSXQUIPT|CSXNAPG|CSXNAPT|CSXQUISG|CSXQUIST|CSXSLTSG|CSXSLTST|CSXNASG|CSXNAST|CSXTSEQ|CSXCHOOD|CSXSBOD|CSXSMKOD|CSXLEAOD|CSXSOAOD|CSXGRAOD|CSXONOD|CSXNGSOD|CSXSLTRT|CSXSLTRG|CSXNART|CSXNARG|CSAEFFRT|SDDSRVYR|RIDSTATR|RIAGENDR|RIDAGEYR|RIDAGEMN|RIDRETH1|RIDRETH3|RIDEXMON|RIDEXAGM|DMQMILIZ|DMQADFC|DMDBORN4|DMDCITZN|DMDYRSUS|DMDEDUC3|DMDEDUC2|DMDMARTL|RIDEXPRG|SIALANG|SIAPROXY|SIAINTRP|FIALANG|FIAPROXY|FIAINTRP|MIALANG|MIAPROXY|MIAINTRP|AIALANGA|DMDHHSIZ|DMDFMSIZ|DMDHHSZA|DMDHHSZB|DMDHHSZE|DMDHRGND|DMDHRAGE|DMDHRBR4|DMDHREDU|DMDHRMAR|DMDHSEDU|    WTINT2YR|    WTMEC2YR|SDMVPSU|SDMVSTRA|INDHHIN2|INDFMIN2|INDFMPIR|DPQ010|DPQ020|DPQ030|DPQ040|DPQ050|DPQ060|DPQ070|DPQ080|DPQ090|
# +-----+--------+--------+--------+------+------+------+------+-------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+--------+-----+-----+--------+--------+-------+-------+-----+-----+------+-------+------+------+-------+-------+-------+-------+--------+--------+-------+-------+-------+-------+--------+--------+--------+------+------+------+------+------+------+------+------+------+-------+-------+--------+--------+-------+--------+-------+--------+-------+--------+-------+--------+-------+--------+-------+--------+-------+--------+--------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+------+------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+--------+--------+-------+-------+--------+--------+--------+--------+-------+-------+-------+--------+-------+--------+--------+--------+--------+-------+--------+--------+--------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+-------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+------------+------------+-------+--------+--------+--------+--------+------+------+------+------+------+------+------+------+------+
# |80900|       1|     974|    null|  null|     1|     5|    76|      1|     1|   140|   112|    70|     2|   116|    72|     2|   116|    76|     2|  null|  null|  null|       1| 84.8| null|    null|    null|   null|   null|162.3| null|  32.2|   null|  37.8|  null|   38.2|   null|   36.1|   null|   106.8|    null|   26.1|   26.3|   null|   null|    26.2|    null|       1|     2|  null|     2|  null|  null|     2|  null|  null|     1|      1|      1|       2|       1|   33.3|       1|   34.4|       1|   34.0|       1|   34.6|       1|   34.4|       1|   34.7|       1|   69.1|       1|       1|     2|      4|      2|      2|      2|      2|      2|      2|      2|      2|      2|      2|      2|      2|      2|      2|      4|      4|      2|      2|      2|      2|      2|      2|      2|      2|      2|      2|      2|      2|      2|      2|      4|       Z|       Z|       Z|       Z|       S|       S|       S|       S|       S|       S|       Z|       Z|       Z|       Z|       Z|       Z|       Z|       Z|       S|       S|       S|       S|       S|       S|       S|       Z|       Z|       Z|      68|   56789|   56789|   56789|    null|    null|    null|    null|    null|    null|   56789|   56789|   56789|      68|   56789|   56789|   56789|      69|    null|    null|    null|    null|    null|    null|    null|   56789|   56789|   56789|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|       1|    null|     2|  null|   null|   null|   null|   null|      1|   null|  null|    15|    36|    20|     8|    53|       7|       2|     32|      2|      56|       2|      43|       1|     17|      1|      B|       2|      1|       3|       3|       1|       2|      3|       2|    null|    null|     28|      1|       1|       8|       2|       1|      66|    null|       1|       1|       2|    null|       1|      1|       1|       1|    null|    null|       4|       3|    null|      1|       2|       2|   null|    null|    null|      1|       2|       2|       1|       3|       3|       0|       0|       3|       1|      60|       1|       3|       2|    null|13833.936085|14908.252501|      2|     107|    null|    null|    null|     0|     0|     0|     0|     0|     0|     0|     0|     0|
# |82949|       1|    1225|    null|  null|     2|     4|    74|      1|     1|   150|   128|    64|     2|   132|    68|     2|   126|    62|     2|  null|  null|  null|       1|116.1| null|    null|    null|   null|   null|186.0| null|  33.6|   null|  43.5|  null|   40.2|   null|   40.6|   null|   116.5|    null|   29.6|   29.6|   null|   null|    29.6|    null|       1|     2|  null|     2|  null|  null|     2|  null|  null|     1|      1|      1|       1|       2|   45.1|       1|   52.5|       1|   47.8|       1|   51.6|       1|   48.1|       1|   52.2|       1|  100.6|       1|       1|     2|      4|      4|      4|      4|      4|      4|      4|      4|      4|      4|      4|      4|      4|      4|      4|      4|      4|      4|      4|      4|      4|      4|      4|      4|      4|      4|      4|      4|      4|      4|      4|      4|       P|       P|       P|       P|       P|       P|       P|       P|       P|       P|       P|       P|       P|       P|       P|       P|       P|       P|       P|       P|       P|       P|       P|       P|       P|       P|       P|       P|    null|    null|    null|    null|    null|    null|    null|    null|    null|    null|    null|    null|    null|    null|    null|    null|    null|    null|    null|    null|    null|    null|    null|    null|    null|    null|    null|    null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|   null|       1|    null|     2|  null|   null|   null|   null|   null|   null|      1|  null|     8|    22|    25|     5|    31|      18|       2|     19|      1|      20|       2|      34|       1|     17|      1|      B|       2|      1|       3|       2|       1|       3|      3|       4|      34|       1|   null|   null|       1|       8|       2|       1|      67|    null|       3|       3|       1|    null|       1|      1|       1|       1|    null|    null|       3|       1|    null|      1|       2|       2|      1|       2|       2|      1|       2|       2|       1|       2|       2|       0|       0|       2|       2|      67|       1|       3|       1|       3|96081.098846|96459.977987|      1|     116|       7|       7|    2.26|     0|     0|     0|     0|     0|     0|     0|     0|     0|
# +-----+--------+--------+--------+------+------+------+------+-------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+--------+-----+-----+--------+--------+-------+-------+-----+-----+------+-------+------+------+-------+-------+-------+-------+--------+--------+-------+-------+-------+-------+--------+--------+--------+------+------+------+------+------+------+------+------+------+-------+-------+--------+--------+-------+--------+-------+--------+-------+--------+-------+--------+-------+--------+-------+--------+-------+--------+--------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+------+------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+--------+--------+-------+-------+--------+--------+--------+--------+-------+-------+-------+--------+-------+--------+--------+--------+--------+-------+--------+--------+--------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+-------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+------------+------------+-------+--------+--------+--------+--------+------+------+------+------+------+------+------+------+------+
# only showing top 2 rows


In [42]:
# 2.2 reduce to important cols
# this should add all of the columns into a column and then delete the old columns, leaving one summed collumn

dfMedsOldVets = vetDFolder_noNull\
.withColumn('totalMentalHealthProblems', (vetDFolder_noNull.DPQ020 + \
                                                       vetDFolder_noNull.DPQ030 + vetDFolder_noNull.DPQ040 + vetDFolder_noNull.DPQ050 + \
                                                       vetDFolder_noNull.DPQ060 + vetDFolder_noNull.DPQ070 + vetDFolder_noNull.DPQ080 + \
                                                       vetDFolder_noNull.DPQ090))\
.drop("DPQ020","DPQ030","DPQ040","DPQ050","DPQ060","DPQ070","DPQ080","DPQ090")

dfMedsYungVets = vetDFyounger_noNull\
.withColumn('totalMentalHealthProblems', (vetDFyounger_noNull.DPQ020 + \
                                                       vetDFyounger_noNull.DPQ030 + vetDFyounger_noNull.DPQ040 + vetDFyounger_noNull.DPQ050 + \
                                                       vetDFyounger_noNull.DPQ060 + vetDFyounger_noNull.DPQ070 + vetDFyounger_noNull.DPQ080 + \
                                                       vetDFyounger_noNull.DPQ090))\
.drop("DPQ020","DPQ030","DPQ040","DPQ050","DPQ060","DPQ070","DPQ080","DPQ090")

dfMedsOldCivs = civiDFolder_noNull\
.withColumn('totalMentalHealthProblems', (civiDFolder_noNull.DPQ020 + \
                                                       civiDFolder_noNull.DPQ030 + civiDFolder_noNull.DPQ040 + civiDFolder_noNull.DPQ050 + \
                                                       civiDFolder_noNull.DPQ060 + civiDFolder_noNull.DPQ070 + civiDFolder_noNull.DPQ080 + \
                                                       civiDFolder_noNull.DPQ090))\
.drop("DPQ020","DPQ030","DPQ040","DPQ050","DPQ060","DPQ070","DPQ080","DPQ090")

dfMedsYungCivs = civiDFyounger_noNull\
.withColumn('totalMentalHealthProblems', (civiDFyounger_noNull.DPQ020 + \
                                                       civiDFyounger_noNull.DPQ030 + civiDFyounger_noNull.DPQ040 + civiDFyounger_noNull.DPQ050 + \
                                                       civiDFyounger_noNull.DPQ060 + civiDFyounger_noNull.DPQ070 + civiDFyounger_noNull.DPQ080 + \
                                                       civiDFyounger_noNull.DPQ090))\
.drop("DPQ020","DPQ030","DPQ040","DPQ050","DPQ060","DPQ070","DPQ080","DPQ090")



In [43]:
dfMedsOldVets_clean_total = dfMedsOldVets.select('SEQN','totalMentalHealthProblems')
dfMedsOldVets_clean_individual = vetDFolder_noNull.select('SEQN',"DPQ020","DPQ030","DPQ040","DPQ050","DPQ060","DPQ070","DPQ080","DPQ090").fillna(0)
dfMedsYungVets_clean_total = dfMedsYungVets.select('SEQN','totalMentalHealthProblems')
vetDFyounger_clean_individual = vetDFyounger_noNull.select('SEQN',"DPQ020","DPQ030","DPQ040","DPQ050","DPQ060","DPQ070","DPQ080","DPQ090").fillna(0)

dfMedsOldCivs_clean_total = dfMedsOldCivs.select('SEQN','totalMentalHealthProblems')
civiDFolder_clean_individual = civiDFolder_noNull.select('SEQN',"DPQ020","DPQ030","DPQ040","DPQ050","DPQ060","DPQ070","DPQ080","DPQ090").fillna(0)
dfMedsYungCivs_clean_total = dfMedsYungCivs.select('SEQN','totalMentalHealthProblems')
civiDFyounger_clean_individual = civiDFyounger_noNull.select('SEQN',"DPQ020","DPQ030","DPQ040","DPQ050","DPQ060","DPQ070","DPQ080","DPQ090").fillna(0)

In [44]:
# dfMedsOldVets_clean.show(4)

# +-----+-------------------------+
# | SEQN|totalMentalHealthProblems|
# +-----+-------------------------+
# |80900|                        0|
# |82949|                        0|
# |83684|                        0|
# |79201|                        9|
# +-----+-------------------------+
# only showing top 4 rows

In [45]:
# print dfMedsOldVets_clean_individual.show(3)

# +------+------+------+------+------+------+------+------+
# |DPQ020|DPQ030|DPQ040|DPQ050|DPQ060|DPQ070|DPQ080|DPQ090|
# +------+------+------+------+------+------+------+------+
# |     0|     0|     0|     0|     0|     0|     0|     0|
# |     0|     0|     0|     0|     0|     0|     0|     0|
# |     0|     0|     0|     0|     0|     0|     0|     0|
# +------+------+------+------+------+------+------+------+
# only showing top 3 rows

In [46]:
# dfMedsOldVets_clean_individual.count()
# ---> 386

In [47]:
# "DPQ010","DPQ020","DPQ030","DPQ040","DPQ050","DPQ060","DPQ070","DPQ080","DPQ090")
from pyspark.sql import functions as f

dfsmal = dfMedsOldVets_clean_individual
dfMedsOldVets_clean_individual_sum = dfsmal.select([f.sum(dfsmal.DPQ020).alias("depres") , f.sum(dfsmal.DPQ030).alias("sleepProb") , \
                                                    f.sum(dfsmal.DPQ040).alias("feelTired") , f.sum(dfsmal.DPQ050).alias("eatDisordr") , \
                                                    f.sum(dfsmal.DPQ060).alias("selfDep") , f.sum(dfsmal.DPQ070).alias("troubConcentrat") , \
                                                    f.sum(dfsmal.DPQ080).alias("behavExtremes_speed") , f.sum(dfsmal.DPQ090).alias("suicidal") ])
dfsmal = vetDFyounger_clean_individual
vetDFyounger_clean_individual_sum = dfsmal.select([f.sum(dfsmal.DPQ020).alias("depres") , f.sum(dfsmal.DPQ030).alias("sleepProb") , \
                                                    f.sum(dfsmal.DPQ040).alias("feelTired") , f.sum(dfsmal.DPQ050).alias("eatDisordr") , \
                                                    f.sum(dfsmal.DPQ060).alias("selfDep") , f.sum(dfsmal.DPQ070).alias("troubConcentrat") , \
                                                    f.sum(dfsmal.DPQ080).alias("behavExtremes_speed") , f.sum(dfsmal.DPQ090).alias("suicidal") ])
dfsmal = civiDFolder_clean_individual
civiDFolder_clean_individual_sum = dfsmal.select([f.sum(dfsmal.DPQ020).alias("depres") , f.sum(dfsmal.DPQ030).alias("sleepProb") , \
                                                    f.sum(dfsmal.DPQ040).alias("feelTired") , f.sum(dfsmal.DPQ050).alias("eatDisordr") , \
                                                    f.sum(dfsmal.DPQ060).alias("selfDep") , f.sum(dfsmal.DPQ070).alias("troubConcentrat") , \
                                                    f.sum(dfsmal.DPQ080).alias("behavExtremes_speed") , f.sum(dfsmal.DPQ090).alias("suicidal") ])
dfsmal = civiDFyounger_clean_individual
civiDFyounger_clean_individual_sum = dfsmal.select([f.sum(dfsmal.DPQ020).alias("depres") , f.sum(dfsmal.DPQ030).alias("sleepProb") , \
                                                    f.sum(dfsmal.DPQ040).alias("feelTired") , f.sum(dfsmal.DPQ050).alias("eatDisordr") , \
                                                    f.sum(dfsmal.DPQ060).alias("selfDep") , f.sum(dfsmal.DPQ070).alias("troubConcentrat") , \
                                                    f.sum(dfsmal.DPQ080).alias("behavExtremes_speed") , f.sum(dfsmal.DPQ090).alias("suicidal") ])

# print "Old Vets:"
# dfMedsOldVets_clean_individual_sum.show()
# Old Vets:
# +------+---------+---------+----------+-------+---------------+-------------------+--------+
# |depres|sleepProb|feelTired|eatDisordr|selfDep|troubConcentrat|behavExtremes_speed|suicidal|
# +------+---------+---------+----------+-------+---------------+-------------------+--------+
# |   117|      206|      257|       101|     86|            105|                 66|      27|
# +------+---------+---------+----------+-------+---------------+-------------------+--------+

# print "Young Vets:"
# vetDFyounger_clean_individual_sum.show()
# Young Vets:
# +------+---------+---------+----------+-------+---------------+-------------------+--------+
# |depres|sleepProb|feelTired|eatDisordr|selfDep|troubConcentrat|behavExtremes_speed|suicidal|
# +------+---------+---------+----------+-------+---------------+-------------------+--------+
# |    54|       78|       95|        58|     36|             41|                 29|       3|
# +------+---------+---------+----------+-------+---------------+-------------------+--------+

# print "Old Civis:"
# civiDFolder_clean_individual_sum.show()
# Old Civis:
# +------+---------+---------+----------+-------+---------------+-------------------+--------+
# |depres|sleepProb|feelTired|eatDisordr|selfDep|troubConcentrat|behavExtremes_speed|suicidal|
# +------+---------+---------+----------+-------+---------------+-------------------+--------+
# |   840|     1295|     1513|       796|    595|            628|                431|     125|
# +------+---------+---------+----------+-------+---------------+-------------------+--------+

# print "Young Civis:"
# civiDFyounger_clean_individual_sum.show()
# Young Civis:
# +------+---------+---------+----------+-------+---------------+-------------------+--------+
# |depres|sleepProb|feelTired|eatDisordr|selfDep|troubConcentrat|behavExtremes_speed|suicidal|
# +------+---------+---------+----------+-------+---------------+-------------------+--------+
# |   894|     1587|     2067|      1071|    675|            709|                392|     132|
# +------+---------+---------+----------+-------+---------------+-------------------+--------+

In [48]:
#scratch

# SELECT AVG(column_name)
# FROM table_name;

# print dfMedsOldVets_clean_individual.count()

# print vetDFyounger_clean_individual.count()

# print civiDFolder_clean_individual.count()

# print civiDFyounger_clean_individual.count()

In [49]:
# dfMedsOldVets_clean_individual.describe("DPQ020","DPQ030","DPQ040","DPQ050","DPQ060","DPQ070","DPQ080","DPQ090").show()
# +-------+-------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+
# |summary|             DPQ020|            DPQ030|            DPQ040|            DPQ050|             DPQ060|             DPQ070|             DPQ080|             DPQ090|
# +-------+-------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+
# |  count|                386|               386|               386|               386|                386|                386|                386|                362|
# |   mean|0.30310880829015546| 0.533678756476684|0.6658031088082902|0.2616580310880829|0.22279792746113988|0.27202072538860106|0.17098445595854922|0.07458563535911603|
# | stddev| 0.7306575395341216|0.9508194362920108|0.9478459530659812|0.7738796491218413| 0.7294178042018186| 0.7935418283465443| 0.5782082173533148| 0.3902772217549807|
# |    min|                  0|                 0|                 0|                 0|                  0|                  0|                  0|                  0|
# |    max|                  3|                 3|                 3|                 9|                  9|                  9|                  3|                  3|
# +-------+-------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+

# vetDFyounger_clean_individual.describe("DPQ020","DPQ030","DPQ040","DPQ050","DPQ060","DPQ070","DPQ080","DPQ090").show()
# +-------+-------------------+------------------+------------------+------------------+-------------------+-------------------+------------------+--------------------+
# |summary|             DPQ020|            DPQ030|            DPQ040|            DPQ050|             DPQ060|             DPQ070|            DPQ080|              DPQ090|
# +-------+-------------------+------------------+------------------+------------------+-------------------+-------------------+------------------+--------------------+
# |  count|                137|               137|               137|               137|                137|                137|               137|                 132|
# |   mean|0.39416058394160586|0.5693430656934306|0.6934306569343066|0.4233576642335766|0.26277372262773724|0.29927007299270075|0.2116788321167883|0.022727272727272728|
# | stddev| 0.7892444659067064|0.9377352288261911|1.0258572010735307|0.8199948685028309| 0.6215501611566313| 0.7511262475506717|0.6798643564355812| 0.19402955196768357|
# |    min|                  0|                 0|                 0|                 0|                  0|                  0|                 0|                   0|
# |    max|                  3|                 3|                 3|                 3|                  3|                  3|                 3|                   2|
# +-------+-------------------+------------------+------------------+------------------+-------------------+-------------------+------------------+--------------------+

# civiDFolder_clean_individual.describe("DPQ020","DPQ030","DPQ040","DPQ050","DPQ060","DPQ070","DPQ080","DPQ090").show()
# +-------+------------------+------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+
# |summary|            DPQ020|            DPQ030|            DPQ040|             DPQ050|            DPQ060|            DPQ070|             DPQ080|             DPQ090|
# +-------+------------------+------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+
# |  count|              2017|              2017|              2017|               2017|              2017|              2017|               2017|               1857|
# |   mean|0.4164600892414477|0.6420426375805652|0.7501239464551314|0.39464551313832424|0.2949925632126921|0.3113534952900347|0.21368368864650472|0.06731287022078622|
# | stddev|0.8706449229529122|1.0161285618136693|0.9945442615595778| 0.8054446948852524|0.7866316782752238| 0.770381143093377| 0.6853419702362752|   0.42331428572023|
# |    min|                 0|                 0|                 0|                  0|                 0|                 0|                  0|                  0|
# |    max|                 9|                 7|                 7|                  3|                 9|                 9|                  9|                  9|
# +-------+------------------+------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+

# civiDFyounger_clean_individual.describe("DPQ020","DPQ030","DPQ040","DPQ050","DPQ060","DPQ070","DPQ080","DPQ090").show()
# +-------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+--------------------+
# |summary|            DPQ020|            DPQ030|            DPQ040|            DPQ050|             DPQ060|             DPQ070|             DPQ080|              DPQ090|
# +-------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+--------------------+
# |  count|              3048|              3048|              3048|              3048|               3048|               3048|               3048|                2724|
# |   mean|0.2933070866141732|0.5206692913385826|0.6781496062992126|0.3513779527559055|0.22145669291338582|0.23261154855643046|0.12860892388451445|0.048458149779735685|
# | stddev|0.6660661112169177|0.8916889780389664|0.8847734239581017|0.7579951733586822| 0.5990685068548548|  0.632787139712549| 0.4680768430785099|  0.3305941880325175|
# |    min|                 0|                 0|                 0|                 0|                  0|                  0|                  0|                   0|
# |    max|                 7|                 3|                 3|                 9|                  3|                  3|                  3|                   9|
# +-------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+--------------------+

# |    max|                 9|                 7|                 7|                  9|                  9                   9                   9                    9

In [50]:
print "old vet mean"
oldVetMean = [0.3031,0.5336, 0.6658, 0.2616, 0.2227, 0.2720, 0.17098, 0.07458]
print oldVetMean
print "young vet mean"
youngVetMean = [0.3941,0.56934,0.6934,0.4233,0.2627,0.2992,0.21167,0.0227]
print youngVetMean
print "old civ mean"
oldCivMean = (0.4164,0.6420,0.7501,0.394645,0.29499,0.3113,0.21368,0.06731)
print oldCivMean
print "young civ mean"
youngCivMean = [0.29330,0.52066,0.6781,0.35137,0.2214,0.2326,0.12860,0.04845]
print youngCivMean

old vet mean
[0.3031, 0.5336, 0.6658, 0.2616, 0.2227, 0.272, 0.17098, 0.07458]
young vet mean
[0.3941, 0.56934, 0.6934, 0.4233, 0.2627, 0.2992, 0.21167, 0.0227]
old civ mean
(0.4164, 0.642, 0.7501, 0.394645, 0.29499, 0.3113, 0.21368, 0.06731)
young civ mean
[0.2933, 0.52066, 0.6781, 0.35137, 0.2214, 0.2326, 0.1286, 0.04845]


In [51]:
from pyspark.sql import functions as f
# 2.2 reduce to important cols
# this should add all of the columns into a column and then delete the old columns, leaving one summed collumn

vetDFolder_clean_normalized = dfMedsOldVets_clean_individual\
.withColumn('DPQ020N', (dfMedsOldVets_clean_individual.DPQ020 / 3.)) \
.withColumn('DPQ030N', (dfMedsOldVets_clean_individual.DPQ030 / 3.)) \
.withColumn('DPQ040N', (dfMedsOldVets_clean_individual.DPQ040 / 3.)) \
.withColumn('DPQ050N', (dfMedsOldVets_clean_individual.DPQ050 / 9.)) \
.withColumn('DPQ060N', (dfMedsOldVets_clean_individual.DPQ060 / 9.)) \
.withColumn('DPQ070N', (dfMedsOldVets_clean_individual.DPQ070 / 9.)) \
.withColumn('DPQ080N', (dfMedsOldVets_clean_individual.DPQ080 / 3.)) \
.withColumn('DPQ090N', (dfMedsOldVets_clean_individual.DPQ090 / 3.)).drop("DPQ020","DPQ030","DPQ040","DPQ050","DPQ060","DPQ070","DPQ080","DPQ090")
# vetDFolder_clean_normalized.show(2)
# +-----+-------+-------+-------+-------+-------+-------+-------+-------+
# | SEQN|DPQ020N|DPQ030N|DPQ040N|DPQ050N|DPQ060N|DPQ070N|DPQ080N|DPQ090N|
# +-----+-------+-------+-------+-------+-------+-------+-------+-------+
# |82949|    0.0|    0.0|    0.0|    0.0|    0.0|    0.0|    0.0|    0.0|
# |80900|    0.0|    0.0|    0.0|    0.0|    0.0|    0.0|    0.0|    0.0|
# +-----+-------+-------+-------+-------+-------+-------+-------+-------+
# only showing top 2 rows


vetDFyounger_clean_normalized = vetDFyounger_clean_individual\
.withColumn('DPQ020N', (vetDFyounger_clean_individual.DPQ020 / 3.)) \
.withColumn('DPQ030N', (vetDFyounger_clean_individual.DPQ030 / 3.)) \
.withColumn('DPQ040N', (vetDFyounger_clean_individual.DPQ040 / 3.)) \
.withColumn('DPQ050N', (vetDFyounger_clean_individual.DPQ050 / 3.)) \
.withColumn('DPQ060N', (vetDFyounger_clean_individual.DPQ060 / 3.)) \
.withColumn('DPQ070N', (vetDFyounger_clean_individual.DPQ070 / 3.)) \
.withColumn('DPQ080N', (vetDFyounger_clean_individual.DPQ080 / 3.)) \
.withColumn('DPQ090N', (vetDFyounger_clean_individual.DPQ090 / 2.)).drop("DPQ020","DPQ030","DPQ040","DPQ050","DPQ060","DPQ070","DPQ080","DPQ090")
# vetDFyounger_clean_normalized.show(2)
# +-----+-------+-------+-------+-------+-------+-------+-------+-------+
# | SEQN|DPQ020N|DPQ030N|DPQ040N|DPQ050N|DPQ060N|DPQ070N|DPQ080N|DPQ090N|
# +-----+-------+-------+-------+-------+-------+-------+-------+-------+
# |82359|    0.0|    0.0|    0.0|    0.0|    0.0|    0.0|    0.0|    0.0|
# |77095|    0.0|    0.0|    0.0|    0.0|    0.0|    0.0|    0.0|    0.0|
# +-----+-------+-------+-------+-------+-------+-------+-------+-------+
# only showing top 2 rows

civiDFolder_clean_normalized = civiDFolder_clean_individual\
.withColumn('DPQ020N', (civiDFolder_clean_individual.DPQ020 / 9.)) \
.withColumn('DPQ030N', (civiDFolder_clean_individual.DPQ030 / 7.)) \
.withColumn('DPQ040N', (civiDFolder_clean_individual.DPQ040 / 7.)) \
.withColumn('DPQ050N', (civiDFolder_clean_individual.DPQ050 / 3.)) \
.withColumn('DPQ060N', (civiDFolder_clean_individual.DPQ060 / 9.)) \
.withColumn('DPQ070N', (civiDFolder_clean_individual.DPQ070 / 9.)) \
.withColumn('DPQ080N', (civiDFolder_clean_individual.DPQ080 / 9.)) \
.withColumn('DPQ090N', (civiDFolder_clean_individual.DPQ090 / 9.)).drop("DPQ020","DPQ030","DPQ040","DPQ050","DPQ060","DPQ070","DPQ080","DPQ090")
# civiDFolder_clean_normalized.show(2)
# +-----+-------+-------------------+-------------------+-------+------------------+-------+------------------+-------+
# | SEQN|DPQ020N|            DPQ030N|            DPQ040N|DPQ050N|           DPQ060N|DPQ070N|           DPQ080N|DPQ090N|
# +-----+-------+-------------------+-------------------+-------+------------------+-------+------------------+-------+
# |73723|    0.0|0.14285714285714285|0.14285714285714285|    0.0|               0.0|    0.0|0.1111111111111111|    0.0|
# |75860|    0.0|                0.0|0.14285714285714285|    0.0|0.1111111111111111|    0.0|               0.0|    0.0|
# +-----+-------+-------------------+-------------------+-------+------------------+-------+------------------+-------+
# only showing top 2 rows

civiDFyounger_clean_normalized = civiDFyounger_clean_individual\
.withColumn('DPQ020N', (civiDFyounger_clean_individual.DPQ020 / 7.)) \
.withColumn('DPQ030N', (civiDFyounger_clean_individual.DPQ030 / 3.)) \
.withColumn('DPQ040N', (civiDFyounger_clean_individual.DPQ040 / 3.)) \
.withColumn('DPQ050N', (civiDFyounger_clean_individual.DPQ050 / 9.)) \
.withColumn('DPQ060N', (civiDFyounger_clean_individual.DPQ060 / 3.)) \
.withColumn('DPQ070N', (civiDFyounger_clean_individual.DPQ070 / 3.)) \
.withColumn('DPQ080N', (civiDFyounger_clean_individual.DPQ080 / 3.)) \
.withColumn('DPQ090N', (civiDFyounger_clean_individual.DPQ090 / 9.)).drop("DPQ020","DPQ030","DPQ040","DPQ050","DPQ060","DPQ070","DPQ080","DPQ090")
# civiDFyounger_clean_normalized.show(2)
# +-----+-------+-------+------------------+------------------+-------+-------+-------+-------+
# | SEQN|DPQ020N|DPQ030N|           DPQ040N|           DPQ050N|DPQ060N|DPQ070N|DPQ080N|DPQ090N|
# +-----+-------+-------+------------------+------------------+-------+-------+-------+-------+
# |74498|    0.0|    0.0|0.3333333333333333|0.1111111111111111|    0.0|    0.0|    0.0|    0.0|
# |74551|    0.0|    0.0|0.3333333333333333|               0.0|    0.0|    0.0|    0.0|    0.0|
# +-----+-------+-------+------------------+------------------+-------+-------+-------+-------+
# only showing top 2 rows

In [52]:
#scratch

# from pyspark.ml.linalg import Vectors
# test = civiDFyounger_clean_normalized\
# .withColumn('test', ([civiDFyounger_clean_normalized.DPQ020N,civiDFyounger_clean_normalized.DPQ030N,civiDFyounger_clean_normalized.DPQ040N,civiDFyounger_clean_normalized.DPQ050N,civiDFyounger_clean_normalized.DPQ060N,civiDFyounger_clean_normalized.DPQ070N,civiDFyounger_clean_normalized.DPQ080N,civiDFyounger_clean_normalized.DPQ090N]))
# test.show()

# dimensionality reduction


In [53]:
from sklearn.decomposition import PCA as PCA1
pca = PCA1(n_components=2)

In [54]:
df6 = df6.fillna(0)
from pyspark.ml.feature import PCA as PCA2
from pyspark.ml.linalg import Vectors

In [55]:
filterningCols_med = ["SEQN","DPQ020","DPQ030","DPQ040","DPQ050","DPQ060","DPQ070","DPQ080","DPQ090"]
df_med_master = df6.select([column for column in df6.columns if column in filterningCols_med]).na.fill(0).rdd.map(list)\
.map(lambda x: (x[0], Vectors.dense(x[1:]))).cache()
print df_med_master.take(2)

[(73557, DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])), (73558, DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]))]


In [100]:
# import pyspark.ml.feature
from pyspark.ml.feature import PCA as PCA2
from pyspark.ml.linalg import Vectors
# from pyspark.mllib.linalg.distributed import RowMatrix

# data = [(Vectors.dense([2.0, 0.0, 3.0, 4.0, 5.0]),),
#         (Vectors.dense([4.0, 0.0, 0.0, 6.0, 7.0]),),
#        (Vectors.dense([4.0, 0.0, 0.0, 6.0, 7.0]),)]
# print data[0:2]
data = df_med_master
# print data[0:2]
dr_df = ss.createDataFrame(data, ["id","features"])
# print 'alive'
dr_df.show(1)
pca = PCA2(k=3, inputCol="features", outputCol="pcaFeatures")


#---> troublesome code for PCA
model = pca.fit(dr_df)
print ' alive'
result = model.transform(dr_df).select('id',"pcaFeatures")
result.show(truncate=False)

+-----+--------------------+
|   id|            features|
+-----+--------------------+
|73557|[0.0,0.0,0.0,0.0,...|
+-----+--------------------+
only showing top 1 row

 alive
+-----+-------------------------------------------------------------+
|id   |pcaFeatures                                                  |
+-----+-------------------------------------------------------------+
|73557|[0.0,0.0,0.0]                                                |
|73558|[0.0,0.0,0.0]                                                |
|73559|[0.0,0.0,0.0]                                                |
|73560|[0.0,0.0,0.0]                                                |
|73561|[-3.046715548517044,0.11402268151574835,-2.4147302639021935] |
|73562|[-6.299060794667952,0.29375429310989465,0.15586500184431606] |
|73563|[0.0,0.0,0.0]                                                |
|73564|[-0.8920063503859061,0.16925455109708393,-0.6671492016237033]|
|73565|[0.0,0.0,0.0]                                  

# pairing dimensionality to potential three different features

In [57]:
# the format for the regressions need to be (feature, [reduced dimensionality])

In [58]:
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.regression import LabeledPoint

In [59]:
#label:
# ID
#features:
#highestGradeSchool| *
#totalPeopleInHouse| *
#totalKids_1| 
# bloodPressure_systolic|
# bloodPressure_dyastolic| 
# BMI/ *

# vetDFOlder_DimensionReduction.show()
# vetDFYounger_DimensionReduction.show()
# civiDFOlder_DimensionReduction.show()
# civiDFYounger_DimensionReduction.show()

In [60]:
#feature 1 for testing == ["highestGradeSchool"]
#scale = 1--5, 1 being kindergarden, 5 being college
#goal is to predict if the person will complete college or not given their mental health status 
# --> >4 == 1 & <4 == 0 (with 1 being yes)
featureToGet = ["ID","highestGradeSchool"]

def normGrade(rdd):
    pId = rdd[0]
    grade = rdd[1]
    retGrade = 0
    if grade > 4:
        retGrade = 1
    else:
        retGrade = 0
    return [pId, retGrade]


#1
vetDFOlder_DimensionReduction_gS = vetDFOlder_DimensionReduction.select([column for column in vetDFOlder_DimensionReduction.columns if column in featureToGet]).na.fill(0).rdd.map(list).map(normGrade)#.map(lambda x: (x[0], [x[1:]])).cache()
# print vetDFOlder_DimensionReduction_gS.take(4) # --> [[79220, 3], [82582, 1], [83250, 4], [78700, 5]] --> [[79220, 0], [82582, 0], [83250, 0], [78700, 1]]
vetDFOlder_DimensionReduction_gS_df = ss.createDataFrame(vetDFOlder_DimensionReduction_gS, ["id","normGrade"])
# vetDFOlder_DimensionReduction_gS_df.show(4)
# +-----+---------+
# |   id|normGrade|
# +-----+---------+
# |79220|        0|
# |82582|        0|
# |83250|        0|
# |78700|        1|
# +-----+---------+
# only showing top 4 rows


#2
vetDFYounger_DimensionReduction_gS = vetDFYounger_DimensionReduction.select([column for column in vetDFYounger_DimensionReduction.columns if column in featureToGet]).na.fill(0).rdd.map(list).map(normGrade)#.map(lambda x: (x[0], [x[1:]])).cache()
# print vetDFYounger_DimensionReduction_gS.take(2) --> [[83291, 4], [82500, 5]]
vetDFYounger_DimensionReduction_gS_df = ss.createDataFrame(vetDFYounger_DimensionReduction_gS, ["id","normGrade"])
# vetDFYounger_DimensionReduction_gS_df.show(4)

#3
civiDFOlder_DimensionReduction_gS = civiDFOlder_DimensionReduction.select([column for column in civiDFOlder_DimensionReduction.columns if column in featureToGet]).na.fill(0).rdd.map(list).map(normGrade)#.map(lambda x: (x[0], [x[1:]])).cache()
# print civiDFOlder_DimensionReduction_gS.take(2)
civiDFOlder_DimensionReduction_gS_df = ss.createDataFrame(civiDFOlder_DimensionReduction_gS, ["id","normGrade"])
# civiDFOlder_DimensionReduction_gS_df.show(4)

#4 
civiDFYounger_DimensionReduction_gS = civiDFYounger_DimensionReduction.select([column for column in civiDFYounger_DimensionReduction.columns if column in featureToGet]).na.fill(0).rdd.map(list).map(normGrade)#.map(lambda x: (x[0], [x[1:]])).cache()
# print civiDFYounger_DimensionReduction_gS.take(2)
civiDFYounger_DimensionReduction_gS_df = ss.createDataFrame(civiDFYounger_DimensionReduction_gS, ["id","normGrade"])
# civiDFYounger_DimensionReduction_gS_df.show(4)

In [61]:
#feature 2 for testing == ["BMIIIII"]
# 0 - 18.5 = underweight, 18.5 - 25 = normal, 25 - 30 = overweight, 30 + = obese 
# if it's normal == 1, otherwise = 0, == BMI problem

featureToGet = ["ID","BMI"]

def normBMI(rdd):
    pId = rdd[0]
    bmi = rdd[1]
    retBmi = 0
    if (bmi > 18.5) and (bmi < 25.1):
        retBmi = 1
    else:
        retBmi = 0
    return [pId, retBmi]


#1
vetDFOlder_DimensionReduction_BMI = vetDFOlder_DimensionReduction.select([column for column in vetDFOlder_DimensionReduction.columns if column in featureToGet]).na.fill(0).rdd.map(list).map(normBMI)#.map(lambda x: (x[0], [x[1:]])).cache()
# print vetDFOlder_DimensionReduction_gS.take(4) # --> [[79220, 3], [82582, 1], [83250, 4], [78700, 5]] --> [[79220, 0], [82582, 0], [83250, 0], [78700, 1]]
vetDFOlder_DimensionReduction_BMI_df = ss.createDataFrame(vetDFOlder_DimensionReduction_BMI, ["id","normBMI"])
# vetDFOlder_DimensionReduction_BMI_df.show(4)
# +-----+-------+
# |   id|normBMI|
# +-----+-------+
# |79220|      0|
# |82582|      0|
# |83250|      0|
# |78700|      1|
# +-----+-------+
# # only showing top 4 rows


#2
vetDFYounger_DimensionReduction_BMI = vetDFYounger_DimensionReduction.select([column for column in vetDFYounger_DimensionReduction.columns if column in featureToGet]).na.fill(0).rdd.map(list).map(normBMI)#.map(lambda x: (x[0], [x[1:]])).cache()
# print vetDFYounger_DimensionReduction_gS.take(2) --> [[83291, 4], [82500, 5]]
vetDFYounger_DimensionReduction_BMI_df = ss.createDataFrame(vetDFYounger_DimensionReduction_BMI, ["id","normBMI"])
# vetDFYounger_DimensionReduction_BMI_df.show(4)

#3
civiDFOlder_DimensionReduction_BMI = civiDFOlder_DimensionReduction.select([column for column in civiDFOlder_DimensionReduction.columns if column in featureToGet]).na.fill(0).rdd.map(list).map(normBMI)#.map(lambda x: (x[0], [x[1:]])).cache()
# print civiDFOlder_DimensionReduction_gS.take(2)
civiDFOlder_DimensionReduction_BMI_df = ss.createDataFrame(civiDFOlder_DimensionReduction_BMI, ["id","normBMI"])
# civiDFOlder_DimensionReduction_BMI_df.show(4)

#4 
civiDFYounger_DimensionReduction_BMI = civiDFYounger_DimensionReduction.select([column for column in civiDFYounger_DimensionReduction.columns if column in featureToGet]).na.fill(0).rdd.map(list).map(normBMI)#.map(lambda x: (x[0], [x[1:]])).cache()
# print civiDFYounger_DimensionReduction_gS.take(2)
civiDFYounger_DimensionReduction_BMI_df = ss.createDataFrame(civiDFYounger_DimensionReduction_BMI, ["id","normBMI"])
# civiDFYounger_DimensionReduction_BMI_df.show(4)



In [62]:
#feature 3 for testing == ["totalPeopleInHouse"]

featureToGet = ["ID","totalPeopleInHouse"]

def normTotPeeps(rdd):
    pId = rdd[0]
    peeps = rdd[1]
    retPeeps = 0
    if peeps > 1:
        retPeeps = 1
    else:
        retBmi = 0
    return [pId, retPeeps]


#1
vetDFOlder_DimensionReduction_peeps = vetDFOlder_DimensionReduction.select([column for column in vetDFOlder_DimensionReduction.columns if column in featureToGet]).na.fill(0).rdd.map(list).map(normTotPeeps)#.map(lambda x: (x[0], [x[1:]])).cache()
# print vetDFOlder_DimensionReduction_gS.take(4) # --> [[79220, 3], [82582, 1], [83250, 4], [78700, 5]] --> [[79220, 0], [82582, 0], [83250, 0], [78700, 1]]
vetDFOlder_DimensionReduction_peeps_df = ss.createDataFrame(vetDFOlder_DimensionReduction_peeps, ["id","normTotPeeps"])
# vetDFOlder_DimensionReduction_peeps_df.show(4)
# +-----+-------+
# |   id|normBMI|
# +-----+-------+
# |79220|      0|
# |82582|      0|
# |83250|      0|
# |78700|      1|
# +-----+-------+
# # only showing top 4 rows


#2
vetDFYounger_DimensionReduction_peeps = vetDFYounger_DimensionReduction.select([column for column in vetDFYounger_DimensionReduction.columns if column in featureToGet]).na.fill(0).rdd.map(list).map(normTotPeeps)#.map(lambda x: (x[0], [x[1:]])).cache()
# print vetDFYounger_DimensionReduction_gS.take(2) --> [[83291, 4], [82500, 5]]
vetDFYounger_DimensionReduction_peeps_df = ss.createDataFrame(vetDFYounger_DimensionReduction_peeps, ["id","normTotPeeps"])
# vetDFYounger_DimensionReduction_peeps_df.show(4)

#3
civiDFOlder_DimensionReduction_peeps = civiDFOlder_DimensionReduction.select([column for column in civiDFOlder_DimensionReduction.columns if column in featureToGet]).na.fill(0).rdd.map(list).map(normTotPeeps)#.map(lambda x: (x[0], [x[1:]])).cache()
# print civiDFOlder_DimensionReduction_gS.take(2)
civiDFOlder_DimensionReduction_peeps_df = ss.createDataFrame(civiDFOlder_DimensionReduction_peeps, ["id","normTotPeeps"])
# civiDFOlder_DimensionReduction_peeps_df.show(4)

#4 
civiDFYounger_DimensionReduction_peeps = civiDFYounger_DimensionReduction.select([column for column in civiDFYounger_DimensionReduction.columns if column in featureToGet]).na.fill(0).rdd.map(list).map(normTotPeeps)#.map(lambda x: (x[0], [x[1:]])).cache()
# print civiDFYounger_DimensionReduction_gS.take(2)
civiDFYounger_DimensionReduction_peeps_df = ss.createDataFrame(civiDFYounger_DimensionReduction_peeps, ["id","normTotPeeps"])
# civiDFYounger_DimensionReduction_peeps_df.show(4)

# Merge the dfs

In [63]:
# join err thing with result
# df = df3.join(df1, [ "SEQN" ])

#Educ Level
print 'Education Level'
vetDFOlder_Analyse_gS_df = vetDFOlder_DimensionReduction_gS_df.join(result, [ "id" ]).select('normGrade', 'pcaFeatures')
vetDFOlder_Analyse_gS_df.show(2)

vetDFYounger_Analyse_gS_df = vetDFYounger_DimensionReduction_gS_df.join(result, [ "id" ]).select('normGrade', 'pcaFeatures')
# vetDFYounger_Analyse_gS_df.show(2)

civiDFOlder_Analyse_gS_df = civiDFOlder_DimensionReduction_gS_df.join(result, [ "id" ]).select('normGrade', 'pcaFeatures')
# civiDFOlder_Analyse_gS_df.show(2)

civiDFYounger_Analyse_gS_df = civiDFYounger_DimensionReduction_gS_df.join(result, [ "id" ]).select('normGrade', 'pcaFeatures')
# civiDFYounger_Analyse_gS_df.show(2)

#BMI
print 'BMI'
vetDFOlder_Analyse_BMI_df = vetDFOlder_DimensionReduction_BMI_df.join(result, [ "id" ]).select('normBMI', 'pcaFeatures')
vetDFOlder_Analyse_BMI_df.show(2)

vetDFYounger_Analyse_BMI_df = vetDFYounger_DimensionReduction_BMI_df.join(result, [ "id" ]).select('normBMI', 'pcaFeatures')
# vetDFYounger_Analyse_BMI_df.show(2)

civiDFOlder_Analyse_BMI_df = civiDFOlder_DimensionReduction_BMI_df.join(result, [ "id" ]).select('normBMI', 'pcaFeatures')
# civiDFOlder_Analyse_BMI_df.show(2)

civiDFYounger_Analyse_BMI_df = civiDFYounger_DimensionReduction_BMI_df.join(result, [ "id" ]).select('normBMI', 'pcaFeatures')
# civiDFYounger_Analyse_BMI_df.show(2)

#Total Household
print 'Total Household'
vetDFOlder_Analyse_peeps_df = vetDFOlder_DimensionReduction_peeps_df.join(result, [ "id" ]).select('normTotPeeps', 'pcaFeatures')
vetDFOlder_Analyse_peeps_df.show(2)

vetDFYounger_Analyse_peeps_df = vetDFYounger_DimensionReduction_peeps_df.join(result, [ "id" ]).select('normTotPeeps', 'pcaFeatures')
# vetDFYounger_Analyse_peeps_df.show(2)

civiDFOlder_Analyse_peeps_df = civiDFOlder_DimensionReduction_peeps_df.join(result, [ "id" ]).select('normTotPeeps', 'pcaFeatures')
# civiDFOlder_Analyse_peeps_df.show(2)

civiDFYounger_Analyse_peeps_df = civiDFYounger_DimensionReduction_peeps_df.join(result, [ "id" ]).select('normTotPeeps', 'pcaFeatures')
# civiDFYounger_Analyse_peeps_df.show(2)

Education Level
+---------+--------------------+
|normGrade|         pcaFeatures|
+---------+--------------------+
|        0|       [0.0,0.0,0.0]|
|        0|[-2.0804890734159...|
+---------+--------------------+
only showing top 2 rows

BMI
+-------+--------------------+
|normBMI|         pcaFeatures|
+-------+--------------------+
|      0|       [0.0,0.0,0.0]|
|      0|[-2.0804890734159...|
+-------+--------------------+
only showing top 2 rows

Total Household
+------------+--------------------+
|normTotPeeps|         pcaFeatures|
+------------+--------------------+
|           1|       [0.0,0.0,0.0]|
|           1|[-2.0804890734159...|
+------------+--------------------+
only showing top 2 rows



# split the data for testing and training

In [64]:
def makingLabledPoint(cleanLine): #this is only used for training
    l = cleanLine
    ID = l[0]  #this is the feature that we are testing for!!
    features = l[1]
    negOrPosPoint = LabeledPoint(ID,features)
    return negOrPosPoint


# join err thing with result
# df = df3.join(df1, [ "SEQN" ])

#Educ Level
print 'Education Level'
vetDFOlder_Analyse_gS_df_lp_tr   ,   vetDFOlder_Analyse_gS_df_lp_te        = vetDFOlder_Analyse_gS_df.rdd.map(list).map(lambda x: [x[0], list(x[1])]).map(makingLabledPoint).randomSplit(weights=[0.7,0.3], seed=1)
print vetDFOlder_Analyse_gS_df_lp_tr.take(2)
print vetDFOlder_Analyse_gS_df_lp_te.take(2)
vetDFYounger_Analyse_gS_df_lp_tr,vetDFYounger_Analyse_gS_df_lp_te    = vetDFYounger_Analyse_gS_df.rdd.map(list).map(lambda x: [x[0], list(x[1])]).map(makingLabledPoint).randomSplit(weights=[0.7,0.3], seed=1)
civiDFOlder_Analyse_gS_df_lp_tr,civiDFOlder_Analyse_gS_df_lp_te      = civiDFOlder_Analyse_gS_df.rdd.map(list).map(lambda x: [x[0], list(x[1])]).map(makingLabledPoint).randomSplit(weights=[0.7,0.3], seed=1)
civiDFYounger_Analyse_gS_df_lp_tr,civiDFYounger_Analyse_gS_df_lp_te  = civiDFYounger_Analyse_gS_df.rdd.map(list).map(lambda x: [x[0], list(x[1])]).map(makingLabledPoint).randomSplit(weights=[0.7,0.3], seed=1)

# #BMI
print 'BMI'
vetDFOlder_Analyse_BMI_df_lp_tr,vetDFOlder_Analyse_BMI_df_lp_te       = vetDFOlder_Analyse_BMI_df.rdd.map(list).map(lambda x: [x[0], list(x[1])]).map(makingLabledPoint).randomSplit(weights=[0.7,0.3], seed=1)
print vetDFOlder_Analyse_BMI_df_lp_tr.take(2)
print vetDFOlder_Analyse_BMI_df_lp_te.take(2)
vetDFYounger_Analyse_BMI_df_lp_tr,vetDFYounger_Analyse_BMI_df_lp_te   = vetDFYounger_Analyse_BMI_df.rdd.map(list).map(lambda x: [x[0], list(x[1])]).map(makingLabledPoint).randomSplit(weights=[0.7,0.3], seed=1)
civiDFOlder_Analyse_BMI_df_lp_tr,civiDFOlder_Analyse_BMI_df_lp_te     = civiDFOlder_Analyse_BMI_df.rdd.map(list).map(lambda x: [x[0], list(x[1])]).map(makingLabledPoint).randomSplit(weights=[0.7,0.3], seed=1)
civiDFYounger_Analyse_BMI_df_lp_tr,civiDFYounger_Analyse_BMI_df_lp_te = civiDFYounger_Analyse_BMI_df.rdd.map(list).map(lambda x: [x[0], list(x[1])]).map(makingLabledPoint).randomSplit(weights=[0.7,0.3], seed=1)

# #Total Household
print 'Total Household'
vetDFOlder_Analyse_peeps_df_lp_tr,vetDFOlder_Analyse_peeps_df_lp_te   = vetDFOlder_Analyse_peeps_df.rdd.map(list).map(lambda x: [x[0], list(x[1])]).map(makingLabledPoint).randomSplit(weights=[0.7,0.3], seed=1)
print vetDFOlder_Analyse_peeps_df_lp_tr.take(2)
print vetDFOlder_Analyse_peeps_df_lp_te.take(2)
vetDFYounger_Analyse_peeps_df_lp_tr,vetDFYounger_Analyse_peeps_df_lp_te = vetDFYounger_Analyse_peeps_df.rdd.map(list).map(lambda x: [x[0], list(x[1])]).map(makingLabledPoint).randomSplit(weights=[0.7,0.3], seed=1)
civiDFOlder_Analyse_peeps_df_lp_tr,civiDFOlder_Analyse_peeps_df_lp_te  = civiDFOlder_Analyse_peeps_df.rdd.map(list).map(lambda x: [x[0], list(x[1])]).map(makingLabledPoint).randomSplit(weights=[0.7,0.3], seed=1)
civiDFYounger_Analyse_peeps_df_lp_tr,civiDFYounger_Analyse_peeps_df_lp_te = civiDFYounger_Analyse_peeps_df.rdd.map(list).map(lambda x: [x[0], list(x[1])]).map(makingLabledPoint).randomSplit(weights=[0.7,0.3], seed=1)


# t1 = t_all.map(makingLabledPoint)
# print t1.take(1)
# trainForTesting_labeled = trainForTesting.map(makingLabledPoint)
# print trainForTesting_labeled.take(1)

Education Level
[LabeledPoint(0.0, [-2.0804890734159986,-1.7953973643135597,-0.3742292667188327]), LabeledPoint(0.0, [-0.49778294833778247,-0.6344753572995515,0.5716019293133305])]
[LabeledPoint(0.0, [0.0,0.0,0.0]), LabeledPoint(0.0, [-4.057379184873387,0.2310854775494008,-0.1769977049225227])]
BMI
[LabeledPoint(0.0, [-2.0804890734159986,-1.7953973643135597,-0.3742292667188327]), LabeledPoint(0.0, [-0.49778294833778247,-0.6344753572995515,0.5716019293133305])]
[LabeledPoint(0.0, [0.0,0.0,0.0]), LabeledPoint(0.0, [-4.057379184873387,0.2310854775494008,-0.1769977049225227])]
Total Household
[LabeledPoint(1.0, [-2.0804890734159986,-1.7953973643135597,-0.3742292667188327]), LabeledPoint(0.0, [-0.49778294833778247,-0.6344753572995515,0.5716019293133305])]
[LabeledPoint(1.0, [0.0,0.0,0.0]), LabeledPoint(1.0, [-4.057379184873387,0.2310854775494008,-0.1769977049225227])]


# regression models
#1 logistic regression
#2 random forest

In [ ]:
#######
# this is using the same data inputs as the last task - splitting the data

In [65]:
# logistic
# http://spark.apache.org/docs/2.0.0/api/python/pyspark.mllib.html

from math import exp
import warnings
import numpy
from numpy import array
from pyspark import RDD, since
from pyspark.streaming import DStream
from pyspark.mllib.common import callMLlibFunc, _py2java, _java2py
from pyspark.mllib.linalg import DenseVector, SparseVector, _convert_to_vector
from pyspark.mllib.regression import (
    LabeledPoint, LinearModel, _regression_train_wrapper,
    StreamingLinearAlgorithm)
from pyspark.mllib.util import Saveable, Loader, inherit_doc

import sys
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.util import MLUtils

In [ ]:
# vetDFOlder_Analyse_gS_df_lp_tr   ,   vetDFOlder_Analyse_gS_df_lp_te   
# vetDFYounger_Analyse_gS_df_lp_tr,vetDFYounger_Analyse_gS_df_lp_te
# civiDFOlder_Analyse_gS_df_lp_tr,civiDFOlder_Analyse_gS_df_lp_te
# civiDFYounger_Analyse_gS_df_lp_tr,civiDFYounger_Analyse_gS_df_lp_te 
# vetDFOlder_Analyse_BMI_df_lp_tr,vetDFOlder_Analyse_BMI_df_lp_te
# vetDFYounger_Analyse_BMI_df_lp_tr,vetDFYounger_Analyse_BMI_df_lp_te
# civiDFOlder_Analyse_BMI_df_lp_tr,civiDFOlder_Analyse_BMI_df_lp_te
# civiDFYounger_Analyse_BMI_df_lp_tr,civiDFYounger_Analyse_BMI_df_lp_te
# vetDFOlder_Analyse_peeps_df_lp_tr,vetDFOlder_Analyse_peeps_df_lp_te
# vetDFYounger_Analyse_peeps_df_lp_tr,vetDFYounger_Analyse_peeps_df_lp_te
# civiDFOlder_Analyse_peeps_df_lp_tr,civiDFOlder_Analyse_peeps_df_lp_te
# civiDFYounger_Analyse_peeps_df_lp_tr,civiDFYounger_Analyse_peeps_df_lp_te

In [83]:
##Models##
#Education
vetDFOlder_model_gS = LogisticRegressionWithLBFGS.train(vetDFOlder_Analyse_gS_df_lp_tr,\
                                                                    iterations=10, regParam=0.000000001,\
                                                                    regType='l2', intercept=True)

vetDFYounger_model_gS = LogisticRegressionWithLBFGS.train(vetDFYounger_Analyse_gS_df_lp_tr,\
                                                                    iterations=10, regParam=0.000000001,\
                                                                    regType='l2', intercept=True)

civiDFOlder_model_gS = LogisticRegressionWithLBFGS.train(civiDFOlder_Analyse_gS_df_lp_tr,\
                                                                    iterations=10, regParam=0.000000001,\
                                                                    regType='l2', intercept=True)

civiDFYounger_model_gS = LogisticRegressionWithLBFGS.train(civiDFYounger_Analyse_gS_df_lp_tr,\
                                                                    iterations=10, regParam=0.000000001,\
                                                                    regType='l2', intercept=True)

#BMI
vetDFOlder_model_BMI = LogisticRegressionWithLBFGS.train(vetDFOlder_Analyse_BMI_df_lp_tr,\
                                                                    iterations=10, regParam=0.000000001,\
                                                                    regType='l2', intercept=True)

vetDFYounger_model_BMI = LogisticRegressionWithLBFGS.train(vetDFYounger_Analyse_BMI_df_lp_tr,\
                                                                    iterations=10, regParam=0.000000001,\
                                                                    regType='l2', intercept=True)

civiDFOlder_model_BMI = LogisticRegressionWithLBFGS.train(civiDFOlder_Analyse_BMI_df_lp_tr,\
                                                                    iterations=10, regParam=0.000000001,\
                                                                    regType='l2', intercept=True)

civiDFYounger_model_BMI = LogisticRegressionWithLBFGS.train(civiDFYounger_Analyse_BMI_df_lp_tr,\
                                                                    iterations=10, regParam=0.000000001,\
                                                                    regType='l2', intercept=True)
#Household
vetDFOlder_model_peeps = LogisticRegressionWithLBFGS.train(vetDFOlder_Analyse_peeps_df_lp_tr,\
                                                                    iterations=10, regParam=0.000000001,\
                                                                    regType='l2', intercept=True)

vetDFYounger_model_peeps = LogisticRegressionWithLBFGS.train(vetDFYounger_Analyse_peeps_df_lp_tr,\
                                                                    iterations=10, regParam=0.000000001,\
                                                                    regType='l2', intercept=True)

civiDFOlder_model_peeps = LogisticRegressionWithLBFGS.train(civiDFOlder_Analyse_peeps_df_lp_tr,\
                                                                    iterations=10, regParam=0.000000001,\
                                                                    regType='l2', intercept=True)

civiDFYounger_model_peeps = LogisticRegressionWithLBFGS.train(civiDFYounger_Analyse_peeps_df_lp_tr,\
                                                                    iterations=10, regParam=0.000000001,\
                                                                    regType='l2', intercept=True)


In [ ]:
##Models##
#Education
vetDFOlder_model_gS
vetDFYounger_model_gS
civiDFOlder_model_gS 
civiDFYounger_model_gS
#BMI
vetDFOlder_model_BMI 
vetDFYounger_model_BMI
civiDFOlder_model_BMI 
civiDFYounger_model_BMI
#Household
vetDFOlder_model_peeps
vetDFYounger_model_peeps
civiDFOlder_model_peeps 
civiDFYounger_model_peeps

##testing data##
vetDFOlder_Analyse_gS_df_lp_te
vetDFYounger_Analyse_gS_df_lp_te
civiDFOlder_Analyse_gS_df_lp_te
civiDFYounger_Analyse_gS_df_lp_te

vetDFOlder_Analyse_BMI_df_lp_te
vetDFYounger_Analyse_BMI_df_lp_te
civiDFOlder_Analyse_BMI_df_lp_te
civiDFYounger_Analyse_BMI_df_lp_te

vetDFOlder_Analyse_peeps_df_lp_te
vetDFYounger_Analyse_peeps_df_lp_te
civiDFOlder_Analyse_peeps_df_lp_te
civiDFYounger_Analyse_peeps_df_lp_te

In [84]:
print vetDFOlder_model_gS.predict([-0.49778294833778247,-0.6344753572995515,0.5716019293133305])
print vetDFOlder_model_gS.predict([-4.057379184873387,0.2310854775494008,-0.1769977049225227])

0
0


In [90]:
labelsAndPreds = vetDFOlder_Analyse_gS_df_lp_tr.map(lambda p: (p.label, vetDFOlder_model_gS.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(vetDFOlder_Analyse_gS_df_lp_tr.count())
print("Old Vet Educ Training Error = " + str(trainErr))

labelsAndPreds = vetDFYounger_Analyse_gS_df_lp_tr.map(lambda p: (p.label, vetDFYounger_model_gS.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(vetDFYounger_Analyse_gS_df_lp_tr.count())
print("Young Vet Educ Training Error = " + str(trainErr))

labelsAndPreds = civiDFOlder_Analyse_gS_df_lp_tr.map(lambda p: (p.label, civiDFOlder_model_gS.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(civiDFOlder_Analyse_gS_df_lp_tr.count())
print("Old Civ Educ Training Error = " + str(trainErr))

labelsAndPreds = civiDFYounger_Analyse_gS_df_lp_tr.map(lambda p: (p.label, civiDFYounger_model_gS.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(civiDFYounger_Analyse_gS_df_lp_tr.count())
print("Young Civ Educ Training Error = " + str(trainErr))


labelsAndPreds = vetDFOlder_Analyse_BMI_df_lp_tr.map(lambda p: (p.label, vetDFOlder_model_BMI.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(vetDFOlder_Analyse_BMI_df_lp_tr.count())
print("Old Vet BMI Training Error = " + str(trainErr))

labelsAndPreds = vetDFYounger_Analyse_BMI_df_lp_tr.map(lambda p: (p.label, vetDFYounger_model_BMI.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(vetDFYounger_Analyse_BMI_df_lp_tr.count())
print("Young Vet BMI Training Error = " + str(trainErr))

labelsAndPreds = civiDFOlder_Analyse_BMI_df_lp_tr.map(lambda p: (p.label, civiDFOlder_model_BMI.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(civiDFOlder_Analyse_BMI_df_lp_tr.count())
print("Old Civ BMI Training Error = " + str(trainErr))

labelsAndPreds = civiDFYounger_Analyse_BMI_df_lp_tr.map(lambda p: (p.label, civiDFYounger_model_BMI.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(civiDFYounger_Analyse_BMI_df_lp_tr.count())
print("Young Civ BMI Training Error = " + str(trainErr))


labelsAndPreds = vetDFOlder_Analyse_peeps_df_lp_tr.map(lambda p: (p.label, vetDFOlder_model_peeps.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(vetDFOlder_Analyse_peeps_df_lp_tr.count())
print("Old Vet Household Training Error = " + str(trainErr))

labelsAndPreds = vetDFYounger_Analyse_peeps_df_lp_tr.map(lambda p: (p.label, vetDFYounger_model_peeps.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(vetDFYounger_Analyse_peeps_df_lp_tr.count())
print("Young Vet Household Training Error = " + str(trainErr))

labelsAndPreds = civiDFOlder_Analyse_peeps_df_lp_tr.map(lambda p: (p.label, civiDFOlder_model_peeps.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(civiDFOlder_Analyse_peeps_df_lp_tr.count())
print("Old Civ Household Training Error = " + str(trainErr))

labelsAndPreds = civiDFYounger_Analyse_peeps_df_lp_tr.map(lambda p: (p.label, civiDFYounger_model_peeps.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(civiDFYounger_Analyse_peeps_df_lp_tr.count())
print("Young Civ Household Training Error = " + str(trainErr))

#with testing data
# Old Vet Educ Training Error = 0.241071428571
# Young Vet Educ Training Error = 0.325
# Old Civ Educ Training Error = 0.220538720539 *
# Young Civ Educ Training Error = 0.266443701226

# Old Vet BMI Training Error = 0.223214285714 *
# Young Vet BMI Training Error = 0.175 *
# Old Civ BMI Training Error = 0.23063973064
# Young Civ BMI Training Error = 0.324414715719

# Old Vet Household Training Error = 0.241071428571
# Young Vet Household Training Error = 0.225 
# Old Civ Household Training Error = 0.232323232323
# Young Civ Household Training Error = 0.0602006688963*




# Old Vet BMI Training Error = 0.223214285714 *
# Young Vet BMI Training Error = 0.175 *
# Old Civ Educ Training Error = 0.220538720539 *
# Young Civ Household Training Error = 0.0602006688963*





#with training data
# Old Vet Educ Training Error = 0.29197080292
# Young Vet Educ Training Error = 0.309278350515
# Old Civ Educ Training Error = 0.227687983134
# Young Civ Educ Training Error = 0.260344026034

# Old Vet BMI Training Error = 0.248175182482
# Young Vet BMI Training Error = 0.237113402062
# Old Civ BMI Training Error = 0.26212227688
# Young Civ BMI Training Error = 0.314737331474

# Old Vet Household Training Error = 0.222627737226
# Young Vet Household Training Error = 0.113402061856
# Old Civ Household Training Error = 0.231201686578
# Young Civ Household Training Error = 0.068340306834

Old Vet Educ Training Error = 0.29197080292
Young Vet Educ Training Error = 0.309278350515
Old Civ Educ Training Error = 0.227687983134
Young Civ Educ Training Error = 0.260344026034
Old Vet BMI Training Error = 0.248175182482
Young Vet BMI Training Error = 0.237113402062
Old Civ BMI Training Error = 0.26212227688
Young Civ BMI Training Error = 0.314737331474
Old Vet Household Training Error = 0.222627737226
Young Vet Household Training Error = 0.113402061856
Old Civ Household Training Error = 0.231201686578
Young Civ Household Training Error = 0.068340306834


In [91]:
# Old Civ Educ Training Error = 0.220538720539
# Young Vet BMI Training Error = 0.175
# Young Civ Household Training Error = 0.0602006688963


# select most telling feature for each data group
#create testing vectors of mental health - reduced using PCA - of size 20
#get average feature prediction for each data group run on testing vectors
#compare averages between similar data sets
#what vector combinations give what kinds of results? - within each dataset
#comparing vector types and results across every dataset - are there any vector types that give differrent results to veterans vs. civis between old and young
#

In [93]:
#vectors for comparing

oldVetMean = (12233 , DenseVector([0.3031, 0.5336, 0.6658, 0.2616, 0.2227, 0.272, 0.17098, 0.07458]))
print oldVetMean
youngVetMean = (12244 , DenseVector([0.3941, 0.56934, 0.6934, 0.4233, 0.2627, 0.2992, 0.21167, 0.0227]))
print youngVetMean
oldCivMean = (12255 , DenseVector([0.4164, 0.642, 0.7501, 0.394645, 0.29499, 0.3113, 0.21368, 0.06731]))
print oldCivMean
youngCivMean = (12266 , DenseVector([0.2933, 0.52066, 0.6781, 0.35137, 0.2214, 0.2326, 0.1286, 0.04845]))
print youngCivMean

# youngVetMean = [0.3941, 0.56934, 0.6934, 0.4233, 0.2627, 0.2992, 0.21167, 0.0227]
# oldCivMean = [0.4164, 0.642, 0.7501, 0.394645, 0.29499, 0.3113, 0.21368, 0.06731]
# youngCivMean = [0.2933, 0.52066, 0.6781, 0.35137, 0.2214, 0.2326, 0.1286, 0.04845]
# maxAll = [9,7,7,9,9,9,9,9]
# minAll = [0,0,0,0,0,0,0,0]

(12233, DenseVector([0.3031, 0.5336, 0.6658, 0.2616, 0.2227, 0.272, 0.171, 0.0746]))
(12244, DenseVector([0.3941, 0.5693, 0.6934, 0.4233, 0.2627, 0.2992, 0.2117, 0.0227]))
(12255, DenseVector([0.4164, 0.642, 0.7501, 0.3946, 0.295, 0.3113, 0.2137, 0.0673]))
(12266, DenseVector([0.2933, 0.5207, 0.6781, 0.3514, 0.2214, 0.2326, 0.1286, 0.0485]))


In [112]:
data2 = sc.parallelize([oldVetMean,youngVetMean,oldCivMean,youngCivMean])
# print data2.show(6)

In [97]:
dr_df2 = ss.createDataFrame(data2, ["id","features"])
dr_df2.show()

+-----+--------------------+
|   id|            features|
+-----+--------------------+
|12233|[0.3031,0.5336,0....|
|12244|[0.3941,0.56934,0...|
|12255|[0.4164,0.642,0.7...|
|12266|[0.2933,0.52066,0...|
+-----+--------------------+



In [107]:
# result2 = model.transform(dr_df2).select('id',"pcaFeatures")

In [108]:
# result.show(truncate=False)

In [119]:
#dimensionality reduction

[(73557, DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])), (73558, DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]))]

# import pyspark.ml.feature
from pyspark.ml.feature import PCA as PCA2
from pyspark.ml.linalg import Vectors
# from pyspark.mllib.linalg.distributed import RowMatrix

# data = [(Vectors.dense([2.0, 0.0, 3.0, 4.0, 5.0]),),
#         (Vectors.dense([4.0, 0.0, 0.0, 6.0, 7.0]),),
#        (Vectors.dense([4.0, 0.0, 0.0, 6.0, 7.0]),)]
# print data[0:2]
# test = sc.parallelize(df_med_master.collect() + [oldVetMean,youngVetMean,oldCivMean,youngCivMean])
# print  test.take(1)
data = ss.createDataFrame(sc.parallelize(df_med_master.collect() + [oldVetMean,youngVetMean,oldCivMean,youngCivMean]), ['id', 'features'])
print data.show(2)
# dr_df = ss.createDataFrame(data, ["id","features"])
# # print 'alive'
# dr_df.show(1)
# pca = PCA2(k=3, inputCol="features", outputCol="pcaFeatures")


# #---> troublesome code for PCA
# model = pca.fit(dr_df)
# print ' alive'
# result = model.transform(dr_df).select('id',"pcaFeatures")
# result.show(truncate=False)

+-----+--------------------+
|   id|            features|
+-----+--------------------+
|73557|[0.0,0.0,0.0,0.0,...|
|73558|[0.0,0.0,0.0,0.0,...|
+-----+--------------------+
only showing top 2 rows

None


In [ ]:
sc.stop()
ss.stop()

# END
# END

# previous endeavours

# Random Forest

In [ ]:
# random forest
# https://spark.apache.org/docs/2.2.0/ml-classification-regression.html#random-forest-regression
# https://spark.apache.org/docs/2.2.0/api/python/pyspark.ml.html#pyspark.ml.regression.RandomForestRegressor

from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.sql import Row



def makingDenseVector(cleanLine): #this is only used for training
    l = cleanLine
    ID = int(l[0])  #this is the feature that we are testing for!!
    features = l[1]
    negOrPosPoint = [ID,Vectors.dense(features)]
    return negOrPosPoint





testForForest  = ss.createDataFrame(vetDFOlder_Analyse_gS_df.rdd.map(list).map(lambda x: [x[0], list(x[1])]).map(makingDenseVector), ["label","features"])
print testForForest.show(2)
rf = RandomForestRegressor(numTrees=2, maxDepth=2, seed=42)
model = rf.fit(testForForest)
print model.featureImportances


# # test1 = ss.createDataFrame([(Vectors.sparse(-3.04,[float(0.11402268151574835)],[float(-2.4147302639021935)],))], ["features"])
# # test1 = ss.createDataFrame([(Vectors.sparse(1, [0], [1.0]),)], ["features"])
# test1 = ss.createDataFrame([(Vectors.sparse(3,[float(0.11402268151574835)],[float(-2.4147302639021935)],))], ["features"])

# model.transform(test1).head().prediction
# # 0.5


allclose(model.treeWeights, [1.0, 1.0])
print model.transform(model).head().prediction
model.numFeatures
model.trees
print dir(model)
print "\n"
print model.treeWeights

# task 3

#1 cluster based on demogs, then on exams, then cluster on both
#2 take the top 3 participant profiles from each cluster defined by having the smallest disance (perhaps a centroid)
#3 compare the mental health disorders of those top 3 per cluster
#4 if they're similar we can determine that the cluster is some how grouping similar people.
#5 from that we'll choose the most comparable 2 clusters btw vet and civ in order to tell if there are any differences.
#6 Thereby allowing us to make possible suggestions to vetts based off of their similar civ counterparts

# KMeans Clustering (testing mode)

In [ ]:
from pyspark.mllib.clustering import KMeans, KMeansModel
from math import sqrt
# testrdd = civiDFyounger_clean_normalized_list

In [ ]:
import numpy as np

labeledRdd = civiDFyounger_clean_normalized_list.map(lambda x: (x[0], np.array(x[1:]))).cache()

featureRdd = civiDFyounger_clean_normalized_list.map(lambda x:  np.array(x[1:])).cache()

featureData = nonlabeledRdd

print civiDFyounger_clean_normalized_list.take(2)

In [ ]:
print featureRdd.take(2)

In [ ]:
def error(point):
        center = clusters.centers[clusters.predict(point)]
        return sqrt(sum([x**2 for x in (point - center)]))

ks = []
es = []
for k in  [1,2,3,4,5,6,7,10,20]:
    clusters = KMeans.train(featureRdd, k, maxIterations=10, initializationMode="random")
    WSSSE = featureRdd.map(lambda point: error(point)).reduce(lambda x, y: x + y)
    ks.append(k)
    es.append(WSSSE)

In [ ]:
plt.plot(ks,es)
plt.xlabel("K")
plt.ylabel("Error")
plt.grid(True)

In [ ]:
clusters = KMeans.train(featureRdd, 6, maxIterations=10, initializationMode="random")
print clusters.clusterCenters

In [ ]:
print myrdd.take(2)

In [ ]:
print clusters.predict( np.array([0.        , 0.        , 0.33333333, 0.11111111, 0.        , 0.        , 0.        , 0.        ] ))

In [ ]:
predictions = labeledRdd.map( lambda kv: (clusters.predict( kv[1]) , kv[0]) )
print predictions.take(10)

In [ ]:
for e in predictions.groupByKey().collect():
    print "\nNew cluster:", clusters.centers[e[0]]
    print [v for v in e[1]]

In [ ]:
model = clusters.fit(featureData)

In [ ]:
# clusters = KMeans().setK(2).setSeed(1)
# model = kmeans.fit(dataset)

In [1]:
from pyspark.ml.clustering import KMeans

# Loads data.
# dataset = spark.read.format("libsvm").load("data/mllib/sample_kmeans_data.txt")

# Trains a k-means model.
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(dataset)

# Evaluate clustering by computing Within Set Sum of Squared Errors.
wssse = model.computeCost(dataset)
print("Within Set Sum of Squared Errors = " + str(wssse))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

ImportError: No module named pyspark.ml.clustering

In [138]:
# civiDFyounger_clean_normalized_list = civiDFyounger_clean_normalized.rdd.map(list)
# print civiDFyounger_clean_normalized_list.take(1)

# civiDFyounger_clean_normalized_list_Labled = civiDFyounger_clean_normalized_list.map(lambda x: (x[0], x[1:])).cache()
# print civiDFyounger_clean_normalized_list_Labled.take(2)

# cSchema = StructType([StructField("WordList", ArrayType(StringType()))])

# import random
# print len(testteset)
# listSizeReduction = random.sample(testteset, 100)
# print listSizeReduction[0]
# print len(listSizeReduction)

In [ ]:
# def turingToArray(rdd):
#     d = line.split(",")
#     key = d[0]
#     value = d[1:]
#     value = np.array([float(v) for v in value])

#scratch
#mymyrdd =  sc.parallelize(myrdd.take(7))
# df.fillna(0)
# print mymyrdd.take(7)

In [ ]:
#the following was an attempt to merge all of the file into one big file called df
#it worked, but it ran too slowly

# df = df2.join(df3) #.join(df4)#.join(df4).join(df5).join(df6)
# df = df3.join(df1, [ "SEQN" ]).join(df2, [ "SEQN" ])\
#         .join(df4, [ "SEQN" ]).join(df6, [ "SEQN" ])
'''
    #not using medication becaus it has multiples of same
''' 
# newThing = df4.join(df5, [ "SEQN" ])#.join(df5, [ "SEQN" ])

# df.join([df3,df1], join='SEQN')
# join(other, on=None, how=None)

# df = df.dropDuplicates(subset=None)
# print df.count()
# df.createGlobalTempView("merging")
# df.show(n=1, truncate=True) #it works!

In [ ]:
#Attempting to turn the RDD into a Datafile instead of just importing the CSV as a dataframe

# examinationRDD.take(3)
# examinationList = open("/scratch/ISE495/2018_project_02/NHANES/examination.csv").readlines()# print examinationList[0]
# columnNames = examinationList[0].replace('"',"").split(",")
# print columnNames[0:20]
# numCols = len(columnNames)

# print len(examinationList[1].split(','))
# data4types = examinationList[2].split(",")
# print data4types[:]
# # print type(dataTypes[0])x

# print len(data4types)

# stringTest = "ima string"
# intTest = 2
# floatTest = 4.0
# colDataTypes = []
# for x in data4types:
##     print type(x)
#     if type(x) == type(4634.21):
#         colDataTypes.append("FloatType()")
#     elif type(x) == type(2):
#         colDataTypes.append("IntegerType()")
#     elif type(x) == type("ima string"):
#         colDataTypes.append("StringType()")

# print colDataTypes
# print len(colDataTypes)
# print dataTypes

# from pyspark.sql.types import *

# fields = [StructField("cuisineName", StringType(), True),
#          StructField("recipeID", IntegerType(), False),
#          StructField("ingredient", StringType(), True)]
# schema = StructType(fields)

# print schema
# print type(schema)

# recipeDF2 = ss.createDataFrame(recipesRDD, schema)
# print recipeDF2.show()
# print "\n\n"
# print type(recipeDF2)

'''
# Implicit DF creation
# Step-1: Think how to deal with first row.
# Step-2: The following kind of statement.
recipeRows = recipesRDD.map(lambda line: Row(cuisineName=line[0],\
                                            recipeID=line[1],\
                                            ingredient=line[2]),\
                                            col3=val[3],\
                                           col4=val[4],\
                                           col5=val[5],\
                                           .......)
# Step-3: Create a DF from Rows data
'''

# l = examinationList[1].split(",")
# print l
# print len(l)

# #l = ["suresh", 5, 5.67]
# types = []
# for i in range(224):
#     types.append(type(l[i]))
# print types

#typeKeys = []
#for i in range(3)

